In [1]:
from osgeo import gdal
import numpy as np
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/


########################################
TCR = "local" #local;global
season = "Annual" #Annual; JJA; DJF
stat = "median" #median;STD
path = "C:/Users/windisch/Desktop/Manuscript_Revised/Final_Revision/BGPvBGC_data" #SPECIFY_DATA_PATH
########################################


"""
ANNUAL
"""
defor_local = np.asarray(Dataset(f"{path}/full_defor_{season}_TCR_{TCR}_{stat}.nc","r")["data"][:])
refor_local = np.asarray(Dataset(f"{path}/full_refor_{season}_TCR_{TCR}_{stat}.nc","r")["data"][:])

defor_dT = np.asarray(Dataset(f"{path}/dT_defor_{season}_TCR_{TCR}_{stat}.nc","r")["data"][:])
refor_dT = np.asarray(Dataset(f"{path}/dT_refor_{season}_TCR_{TCR}_{stat}.nc","r")["data"][:])

defor_dC = np.asarray(Dataset(f"{path}/dC_defor_{season}_TCR_X_{stat}.nc","r")["data"][:])
refor_dC = np.asarray(Dataset(f"{path}/dC_refor_{season}_TCR_X_{stat}.nc","r")["data"][:])
defor_local = np.flip(defor_local,axis=0)
refor_local = np.flip(refor_local,axis=0)
defor_dT = np.flip(defor_dT,axis=0)
refor_dT = np.flip(refor_dT,axis=0)
defor_dC = np.flip(defor_dC,axis=0)
refor_dC = np.flip(refor_dC,axis=0)

"""
LOAD LANDCOVER DATASET
"""
###
#0 OTHER
#1 Tropical rainforest
#41 Temperate continental forest
#42 Temperate oceanic forest
#43 Temperate mountain system
#44 Temperate steppe
#6 Boreal coniferous forest
#61 Boreal mountain system
#62 Boreal tundra woodland
add_zeros = np.zeros((78,4320))


rasterload = gdal.Open("C:/BGP_Assessment/GLC2000/ecoregion_mask_GLC2000_reproj.tif")
ecoregion_mask = np.concatenate((add_zeros,np.array(rasterload.GetRasterBand(1).ReadAsArray())),axis=0)
#ecoregion_mask = np.array(rasterload.GetRasterBand(1).ReadAsArray())
rasterload = None

tropical_mask = (ecoregion_mask==1)
temperate_mask = (ecoregion_mask==42)+(ecoregion_mask==43)+(ecoregion_mask==41)+(ecoregion_mask==44)
temperate_oceanic_mask = (ecoregion_mask==42)
temperate_continental_mask = (ecoregion_mask==41)
boreal_mask = (ecoregion_mask==6)+(ecoregion_mask==61)+(ecoregion_mask==62)




add_zeros = np.zeros((24,4320))

rasterload = gdal.Open("C:/BGP_Assessment/GLC2000/gra_13_biomass_small.tif")
GRA_mask = np.concatenate((add_zeros,np.array(rasterload.GetRasterBand(1).ReadAsArray()),add_zeros),axis=0)
GRA_mask[GRA_mask>0]=1
GRA_mask[GRA_mask<=0]=np.nan


rasterload = None

rasterload = gdal.Open("C:/BGP_Assessment/GLC2000/cro_16_17_18_biomass_small.tif")
CRO_mask = np.concatenate((add_zeros,np.array(rasterload.GetRasterBand(1).ReadAsArray()),add_zeros),axis=0)
CRO_mask[CRO_mask>0]=1
CRO_mask[CRO_mask<=0]=np.nan

rasterload = None

add_zeros = np.zeros((24,4320))

rasterload = gdal.Open("C:/BGP_Assessment/GLC2000/shrub_mask_GLC2000_eq_11_12_15_small.tif")
SHR_mask = np.concatenate((add_zeros,np.array(rasterload.GetRasterBand(1).ReadAsArray()),add_zeros),axis=0)
SHR_mask[SHR_mask>0]=1
SHR_mask[SHR_mask<=0]=np.nan


rasterload = None

"""
AREA
"""

lat_large = np.arange(-90,90,180/np.size(defor_local,0))
lon_large = np.arange(-180,180,360/np.size(defor_local,1))

area_grid = np.zeros_like(defor_local)


#https://gis.stackexchange.com/questions/29734/how-to-calculate-area-of-1-x-1-degree-cells-in-a-raster/29743#29743
for i in range(0,np.size(defor_local,0)-1):
    area_grid[i,:]= (np.sin(lat_large[i+1]*np.pi/180) - np.sin(lat_large[i]*np.pi/180)) * (0.08333333)*np.pi/180 * 63710**2

area_grid[2159,:] = area_grid[0,0]

defor_area = np.nansum(~np.isnan(defor_local)*area_grid)
refor_area = np.nansum(~np.isnan(refor_local)*area_grid)
mean_area = np.nanmean(area_grid)

In [25]:
###Number Calculation
"""
TROPICAL
"""

##MEAN
refor_tropical = np.where(refor_local*tropical_mask!=0,refor_local*tropical_mask,np.nan)
defor_tropical = np.where(defor_local*tropical_mask!=0,defor_local*tropical_mask,np.nan)

refor_tropical_area = np.nansum(~np.isnan(refor_tropical)*area_grid)
defor_tropical_area = np.nansum(~np.isnan(defor_tropical)*area_grid)


total_tropical_mean = (np.nansum(refor_tropical*area_grid)+np.nansum(defor_tropical*area_grid))/(refor_tropical_area+defor_tropical_area)

#RANGE
refor_tropical_area_mask = np.where(~np.isnan(refor_tropical),area_grid,np.nan)
refor_tropical_weighted = area_grid*refor_tropical/np.nanmean(refor_tropical_area_mask)
defor_tropical_area_mask = np.where(~np.isnan(defor_tropical),area_grid,np.nan)
defor_tropical_weighted = area_grid*defor_tropical/np.nanmean(defor_tropical_area_mask)
tropical_std = np.nanstd(np.concatenate((refor_tropical_weighted,defor_tropical_weighted),axis=0))

###dT
dT_refor_tropical = np.where(refor_dT*tropical_mask!=0,refor_dT*tropical_mask,np.nan)
dT_defor_tropical = np.where(defor_dT*tropical_mask!=0,defor_dT*tropical_mask,np.nan)
dT_refor_tropical_weighted = refor_tropical_area_mask*dT_refor_tropical/np.nanmean(refor_tropical_area_mask)
dT_defor_tropical_weighted = defor_tropical_area_mask*dT_defor_tropical/np.nanmean(defor_tropical_area_mask)

dT_tropical_mean = (np.nansum(dT_refor_tropical*area_grid)+np.nansum(dT_defor_tropical*area_grid))/(refor_tropical_area+defor_tropical_area)
dT_tropical_std = np.nanstd(np.concatenate((dT_refor_tropical_weighted,dT_defor_tropical_weighted),axis=0))

###dC
dC_refor_tropical = np.where(refor_dC*tropical_mask!=0,refor_dC*tropical_mask,np.nan)
dC_defor_tropical = np.where(defor_dC*tropical_mask!=0,defor_dC*tropical_mask,np.nan)
dC_refor_tropical_weighted = refor_tropical_area_mask*dC_refor_tropical/np.nanmean(refor_tropical_area_mask)
dC_defor_tropical_weighted = defor_tropical_area_mask*dC_defor_tropical/np.nanmean(defor_tropical_area_mask)

dC_tropical_mean = (np.nansum(dC_refor_tropical*area_grid)+np.nansum(dC_defor_tropical*area_grid))/(refor_tropical_area+defor_tropical_area)


###FRACTION
fraction_refor_tropical = dT_refor_tropical/dC_refor_tropical
fraction_defor_tropical = dT_defor_tropical/dC_defor_tropical
fraction_refor_tropical_weighted = dT_refor_tropical_weighted/dC_refor_tropical_weighted
fraction_defor_tropical_weighted = dT_defor_tropical_weighted/dC_defor_tropical_weighted

fraction_refor_tropical_weighted_abs = np.abs(dT_refor_tropical_weighted)/np.abs(dC_refor_tropical_weighted)
fraction_defor_tropical_weighted_abs = np.abs(dT_defor_tropical_weighted)/np.abs(dC_defor_tropical_weighted)


fraction_tropical_mean = (np.nansum(fraction_refor_tropical*area_grid)+np.nansum(fraction_defor_tropical*area_grid))/(refor_tropical_area+defor_tropical_area)

fraction_tropical_weighted_median = np.nanmedian(np.concatenate((fraction_refor_tropical_weighted,fraction_defor_tropical_weighted),axis=0))
fraction_tropical_weighted_median_abs = np.nanmedian(np.concatenate((fraction_refor_tropical_weighted_abs,fraction_defor_tropical_weighted_abs),axis=0))
fraction_tropical_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_tropical_weighted_abs,fraction_defor_tropical_weighted_abs),axis=0))


#Get rid of outliers
print(np.count_nonzero(np.concatenate((fraction_refor_tropical_weighted_abs,fraction_defor_tropical_weighted_abs),axis=0)>5)/np.count_nonzero(~np.isnan(np.concatenate((fraction_refor_tropical_weighted_abs,fraction_defor_tropical_weighted_abs),axis=0))))
fraction_refor_tropical_weighted[fraction_refor_tropical_weighted_abs>5]=np.nan
fraction_defor_tropical_weighted[fraction_defor_tropical_weighted_abs>5]=np.nan
fraction_refor_tropical_weighted_abs[fraction_refor_tropical_weighted_abs>5]=np.nan
fraction_defor_tropical_weighted_abs[fraction_defor_tropical_weighted_abs>5]=np.nan

fraction_tropical_weighted_mean = np.nanmean(np.concatenate((fraction_refor_tropical_weighted,fraction_defor_tropical_weighted),axis=0))
fraction_tropical_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_tropical_weighted_abs,fraction_defor_tropical_weighted_abs),axis=0))
fraction_tropical_weighted_std_abs = np.nanstd(np.concatenate((fraction_refor_tropical_weighted_abs,fraction_defor_tropical_weighted_abs),axis=0))
fraction_tropical_weighted_std = np.nanstd(np.concatenate((fraction_refor_tropical_weighted,fraction_defor_tropical_weighted),axis=0))
fraction_tropical_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_refor_tropical_weighted,fraction_defor_tropical_weighted),axis=0),5)
fraction_tropical_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_refor_tropical_weighted,fraction_defor_tropical_weighted),axis=0),95)



"""
TEMPERATE
"""
###MEAN
refor_temperate = np.where(refor_local*temperate_mask!=0,refor_local*temperate_mask,np.nan)
defor_temperate = np.where(defor_local*temperate_mask!=0,defor_local*temperate_mask,np.nan)

refor_temperate_area = np.nansum(~np.isnan(refor_temperate)*area_grid)
defor_temperate_area = np.nansum(~np.isnan(defor_temperate)*area_grid)

total_temperate_mean = (np.nansum(refor_temperate*area_grid)+np.nansum(defor_temperate*area_grid))/(refor_temperate_area+defor_temperate_area)


#RANGE
refor_temperate_area_mask = np.where(~np.isnan(refor_temperate),area_grid,np.nan)
refor_temperate_weighted = area_grid*refor_temperate/np.nanmean(refor_temperate_area_mask)
defor_temperate_area_mask = np.where(~np.isnan(defor_temperate),area_grid,np.nan)
defor_temperate_weighted = area_grid*defor_temperate/np.nanmean(defor_temperate_area_mask)
temperate_std = np.nanstd(np.concatenate((refor_temperate_weighted,defor_temperate_weighted),axis=0))

###dT
dT_refor_temperate = np.where(refor_dT*temperate_mask!=0,refor_dT*temperate_mask,np.nan)
dT_defor_temperate = np.where(defor_dT*temperate_mask!=0,defor_dT*temperate_mask,np.nan)
dT_refor_temperate_weighted = refor_temperate_area_mask*dT_refor_temperate/np.nanmean(refor_temperate_area_mask)
dT_defor_temperate_weighted = defor_temperate_area_mask*dT_defor_temperate/np.nanmean(defor_temperate_area_mask)

dT_neg_refor_temperate = np.where(dT_refor_temperate<0,dT_refor_temperate,np.nan)
dT_neg_defor_temperate = np.where(dT_defor_temperate<0,dT_defor_temperate,np.nan)
dT_neg_refor_temperate_area_mask = np.where(~np.isnan(dT_neg_refor_temperate),area_grid,np.nan)
dT_neg_defor_temperate_area_mask = np.where(~np.isnan(dT_neg_defor_temperate),area_grid,np.nan)
dT_neg_refor_temperate_weighted = area_grid*dT_neg_refor_temperate/np.nanmean(dT_neg_refor_temperate_area_mask)
dT_neg_defor_temperate_weighted = area_grid*dT_neg_defor_temperate/np.nanmean(dT_neg_defor_temperate_area_mask)


dT_neg_refor_temperate_area_perc = 100*(np.nansum([dT_refor_temperate<=0]*area_grid)/(refor_temperate_area))
dT_neg_defor_temperate_area_perc = 100*(np.nansum([dT_defor_temperate<=0]*area_grid)/(defor_temperate_area))
dT_neg_temperate_area_perc = 100*(np.nansum([dT_refor_temperate<=0]*area_grid)+np.nansum([dT_defor_temperate<=0]*area_grid))/(refor_temperate_area+defor_temperate_area)


dT_temperate_mean = (np.nansum(dT_refor_temperate*area_grid)+np.nansum(dT_defor_temperate*area_grid))/(refor_temperate_area+defor_temperate_area)
dT_temperate_std = np.nanstd(np.concatenate((dT_refor_temperate_weighted,dT_defor_temperate_weighted),axis=0))

###dC
dC_refor_temperate = np.where(refor_dC*temperate_mask!=0,refor_dC*temperate_mask,np.nan)
dC_defor_temperate = np.where(defor_dC*temperate_mask!=0,defor_dC*temperate_mask,np.nan)
dC_refor_temperate_weighted = refor_temperate_area_mask*dC_refor_temperate/np.nanmean(refor_temperate_area_mask)
dC_defor_temperate_weighted = defor_temperate_area_mask*dC_defor_temperate/np.nanmean(defor_temperate_area_mask)

dC_neg_refor_temperate = np.where(dT_refor_temperate<0,dC_refor_temperate,np.nan)
dC_neg_defor_temperate = np.where(dT_defor_temperate<0,dC_defor_temperate,np.nan)
dC_neg_refor_temperate_area_mask = np.where(~np.isnan(dC_neg_refor_temperate),area_grid,np.nan)
dC_neg_defor_temperate_area_mask = np.where(~np.isnan(dC_neg_defor_temperate),area_grid,np.nan)
dC_neg_refor_temperate_weighted = area_grid*dC_neg_refor_temperate/np.nanmean(dC_neg_refor_temperate_area_mask)
dC_neg_defor_temperate_weighted = area_grid*dC_neg_defor_temperate/np.nanmean(dC_neg_defor_temperate_area_mask)


dC_temperate_mean = (np.nansum(dC_refor_temperate*area_grid)+np.nansum(dC_defor_temperate*area_grid))/(refor_temperate_area+defor_temperate_area)


###FRACTION
fraction_refor_temperate = dT_refor_temperate/dC_refor_temperate
fraction_defor_temperate = dT_defor_temperate/dC_defor_temperate
fraction_refor_temperate_weighted = dT_refor_temperate_weighted/dC_refor_temperate_weighted
fraction_defor_temperate_weighted = dT_defor_temperate_weighted/dC_defor_temperate_weighted
fraction_refor_temperate_weighted_abs = np.abs(dT_refor_temperate_weighted)/np.abs(dC_refor_temperate_weighted)
fraction_defor_temperate_weighted_abs = np.abs(dT_defor_temperate_weighted)/np.abs(dC_defor_temperate_weighted)
fraction_neg_refor_temperate_weighted = dT_neg_refor_temperate_weighted/dC_neg_refor_temperate_weighted
fraction_neg_defor_temperate_weighted = dT_neg_defor_temperate_weighted/dC_neg_defor_temperate_weighted

fraction_temperate_mean = (np.nansum(fraction_refor_temperate*area_grid)+np.nansum(fraction_defor_temperate*area_grid))/(refor_temperate_area+defor_temperate_area)


fraction_temperate_weighted_median = np.nanmedian(np.concatenate((fraction_refor_temperate_weighted,fraction_defor_temperate_weighted),axis=0))
fraction_temperate_weighted_median_abs = np.nanmedian(np.concatenate((fraction_refor_temperate_weighted_abs,fraction_defor_temperate_weighted_abs),axis=0))
fraction_temperate_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_temperate_weighted_abs,fraction_defor_temperate_weighted_abs),axis=0))

#Get rid of outliers
print(np.count_nonzero(np.concatenate((fraction_refor_temperate_weighted_abs,fraction_defor_temperate_weighted_abs),axis=0)>5)/np.count_nonzero(~np.isnan(np.concatenate((fraction_refor_temperate_weighted_abs,fraction_defor_temperate_weighted_abs),axis=0))))
fraction_refor_temperate_weighted[fraction_refor_temperate_weighted_abs>5]=np.nan
fraction_defor_temperate_weighted[fraction_defor_temperate_weighted_abs>5]=np.nan
fraction_refor_temperate_weighted_abs[fraction_refor_temperate_weighted_abs>5]=np.nan
fraction_defor_temperate_weighted_abs[fraction_defor_temperate_weighted_abs>5]=np.nan

fraction_temperate_weighted_mean = np.nanmean(np.concatenate((fraction_refor_temperate_weighted,fraction_defor_temperate_weighted),axis=0))
fraction_temperate_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_temperate_weighted_abs,fraction_defor_temperate_weighted_abs),axis=0))
fraction_temperate_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_refor_temperate_weighted,fraction_defor_temperate_weighted),axis=0),5)
fraction_temperate_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_refor_temperate_weighted,fraction_defor_temperate_weighted),axis=0),95)


fraction_neg_refor_temperate_weighted[fraction_neg_refor_temperate_weighted>5]=5
fraction_neg_defor_temperate_weighted[fraction_neg_defor_temperate_weighted>5]=5
fraction_neg_refor_temperate_weighted_mean = np.nanmean(fraction_neg_refor_temperate_weighted)
fraction_neg_defor_temperate_weighted_mean = np.nanmean(fraction_neg_defor_temperate_weighted)


fraction_neg_temperate_weighted_mean = np.nanmean(np.concatenate((fraction_neg_refor_temperate_weighted,fraction_neg_defor_temperate_weighted),axis=0))
fraction_neg_temperate_weighted_median = np.nanmedian(np.concatenate((fraction_neg_refor_temperate_weighted,fraction_neg_defor_temperate_weighted),axis=0))


fraction_temperate_weighted_std_abs = np.nanstd(np.concatenate((fraction_refor_temperate_weighted_abs,fraction_defor_temperate_weighted_abs),axis=0))
fraction_temperate_weighted_std = np.nanstd(np.concatenate((fraction_refor_temperate_weighted,fraction_defor_temperate_weighted),axis=0))



"""
TEMPERATE_OCEANIC
"""
###MEAN
refor_temperate_oceanic = np.where(refor_local*temperate_oceanic_mask!=0,refor_local*temperate_oceanic_mask,np.nan)
defor_temperate_oceanic = np.where(defor_local*temperate_oceanic_mask!=0,defor_local*temperate_oceanic_mask,np.nan)

refor_temperate_oceanic_area = np.nansum(~np.isnan(refor_temperate_oceanic)*area_grid)
defor_temperate_oceanic_area = np.nansum(~np.isnan(defor_temperate_oceanic)*area_grid)

total_temperate_oceanic_mean = (np.nansum(refor_temperate_oceanic*area_grid)+np.nansum(defor_temperate_oceanic*area_grid))/(refor_temperate_oceanic_area+defor_temperate_oceanic_area)


#RANGE
refor_temperate_oceanic_area_mask = np.where(~np.isnan(refor_temperate_oceanic),area_grid,np.nan)
refor_temperate_oceanic_weighted = area_grid*refor_temperate_oceanic/np.nanmean(refor_temperate_oceanic_area_mask)
defor_temperate_oceanic_area_mask = np.where(~np.isnan(defor_temperate_oceanic),area_grid,np.nan)
defor_temperate_oceanic_weighted = area_grid*defor_temperate_oceanic/np.nanmean(defor_temperate_oceanic_area_mask)
temperate_oceanic_std = np.nanstd(np.concatenate((refor_temperate_oceanic_weighted,defor_temperate_oceanic_weighted),axis=0))

###dT
dT_refor_temperate_oceanic = np.where(refor_dT*temperate_oceanic_mask!=0,refor_dT*temperate_oceanic_mask,np.nan)
dT_defor_temperate_oceanic = np.where(defor_dT*temperate_oceanic_mask!=0,defor_dT*temperate_oceanic_mask,np.nan)
dT_refor_temperate_oceanic_weighted = refor_temperate_oceanic_area_mask*dT_refor_temperate_oceanic/np.nanmean(refor_temperate_oceanic_area_mask)
dT_defor_temperate_oceanic_weighted = defor_temperate_oceanic_area_mask*dT_defor_temperate_oceanic/np.nanmean(defor_temperate_oceanic_area_mask)

dT_neg_refor_temperate_oceanic = np.where(dT_refor_temperate_oceanic<0,dT_refor_temperate_oceanic,np.nan)
dT_neg_defor_temperate_oceanic = np.where(dT_defor_temperate_oceanic<0,dT_defor_temperate_oceanic,np.nan)
dT_neg_refor_temperate_oceanic_area_mask = np.where(~np.isnan(dT_neg_refor_temperate_oceanic),area_grid,np.nan)
dT_neg_defor_temperate_oceanic_area_mask = np.where(~np.isnan(dT_neg_defor_temperate_oceanic),area_grid,np.nan)
dT_neg_refor_temperate_oceanic_weighted = area_grid*dT_neg_refor_temperate_oceanic/np.nanmean(dT_neg_refor_temperate_oceanic_area_mask)
dT_neg_defor_temperate_oceanic_weighted = area_grid*dT_neg_defor_temperate_oceanic/np.nanmean(dT_neg_defor_temperate_oceanic_area_mask)


dT_neg_refor_temperate_oceanic_area_perc = 100*(np.nansum([dT_refor_temperate_oceanic<=0]*area_grid)/(refor_temperate_oceanic_area))
dT_neg_defor_temperate_oceanic_area_perc = 100*(np.nansum([dT_defor_temperate_oceanic<=0]*area_grid)/(defor_temperate_oceanic_area))
dT_neg_temperate_oceanic_area_perc = 100*(np.nansum([dT_refor_temperate_oceanic<=0]*area_grid)+np.nansum([dT_defor_temperate_oceanic<=0]*area_grid))/(refor_temperate_oceanic_area+defor_temperate_oceanic_area)


dT_temperate_oceanic_mean = (np.nansum(dT_refor_temperate_oceanic*area_grid)+np.nansum(dT_defor_temperate_oceanic*area_grid))/(refor_temperate_oceanic_area+defor_temperate_oceanic_area)
dT_temperate_oceanic_std = np.nanstd(np.concatenate((dT_refor_temperate_oceanic_weighted,dT_defor_temperate_oceanic_weighted),axis=0))

###dC
dC_refor_temperate_oceanic = np.where(refor_dC*temperate_oceanic_mask!=0,refor_dC*temperate_oceanic_mask,np.nan)
dC_defor_temperate_oceanic = np.where(defor_dC*temperate_oceanic_mask!=0,defor_dC*temperate_oceanic_mask,np.nan)
dC_refor_temperate_oceanic_weighted = refor_temperate_oceanic_area_mask*dC_refor_temperate_oceanic/np.nanmean(refor_temperate_oceanic_area_mask)
dC_defor_temperate_oceanic_weighted = defor_temperate_oceanic_area_mask*dC_defor_temperate_oceanic/np.nanmean(defor_temperate_oceanic_area_mask)

dC_neg_refor_temperate_oceanic = np.where(dT_refor_temperate_oceanic<0,dC_refor_temperate_oceanic,np.nan)
dC_neg_defor_temperate_oceanic = np.where(dT_defor_temperate_oceanic<0,dC_defor_temperate_oceanic,np.nan)
dC_neg_refor_temperate_oceanic_area_mask = np.where(~np.isnan(dC_neg_refor_temperate_oceanic),area_grid,np.nan)
dC_neg_defor_temperate_oceanic_area_mask = np.where(~np.isnan(dC_neg_defor_temperate_oceanic),area_grid,np.nan)
dC_neg_refor_temperate_oceanic_weighted = area_grid*dC_neg_refor_temperate_oceanic/np.nanmean(dC_neg_refor_temperate_oceanic_area_mask)
dC_neg_defor_temperate_oceanic_weighted = area_grid*dC_neg_defor_temperate_oceanic/np.nanmean(dC_neg_defor_temperate_oceanic_area_mask)


dC_temperate_oceanic_mean = (np.nansum(dC_refor_temperate_oceanic*area_grid)+np.nansum(dC_defor_temperate_oceanic*area_grid))/(refor_temperate_oceanic_area+defor_temperate_oceanic_area)


###FRACTION
fraction_refor_temperate_oceanic = dT_refor_temperate_oceanic/dC_refor_temperate_oceanic
fraction_defor_temperate_oceanic = dT_defor_temperate_oceanic/dC_defor_temperate_oceanic
fraction_refor_temperate_oceanic_weighted = dT_refor_temperate_oceanic_weighted/dC_refor_temperate_oceanic_weighted
fraction_defor_temperate_oceanic_weighted = dT_defor_temperate_oceanic_weighted/dC_defor_temperate_oceanic_weighted
fraction_refor_temperate_oceanic_weighted_abs = np.abs(dT_refor_temperate_oceanic_weighted)/np.abs(dC_refor_temperate_oceanic_weighted)
fraction_defor_temperate_oceanic_weighted_abs = np.abs(dT_defor_temperate_oceanic_weighted)/np.abs(dC_defor_temperate_oceanic_weighted)
fraction_neg_refor_temperate_oceanic_weighted = dT_neg_refor_temperate_oceanic_weighted/dC_neg_refor_temperate_oceanic_weighted
fraction_neg_defor_temperate_oceanic_weighted = dT_neg_defor_temperate_oceanic_weighted/dC_neg_defor_temperate_oceanic_weighted

#fraction_temperate_oceanic_mean = (np.nansum(fraction_refor_temperate_oceanic*area_grid)+np.nansum(fraction_defor_temperate_oceanic*area_grid))/(refor_temperate_oceanic_area+defor_temperate_oceanic_area)


fraction_temperate_oceanic_weighted_median = np.nanmedian(np.concatenate((fraction_refor_temperate_oceanic_weighted,fraction_defor_temperate_oceanic_weighted),axis=0))
fraction_temperate_oceanic_weighted_median_abs = np.nanmedian(np.concatenate((fraction_refor_temperate_oceanic_weighted_abs,fraction_defor_temperate_oceanic_weighted_abs),axis=0))
fraction_temperate_oceanic_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_temperate_oceanic_weighted_abs,fraction_defor_temperate_oceanic_weighted_abs),axis=0))

#Get rid of outliers
print(np.count_nonzero(np.concatenate((fraction_refor_temperate_oceanic_weighted_abs,fraction_defor_temperate_oceanic_weighted_abs),axis=0)>5)/np.count_nonzero(~np.isnan(np.concatenate((fraction_refor_temperate_oceanic_weighted_abs,fraction_defor_temperate_oceanic_weighted_abs),axis=0))))
fraction_refor_temperate_oceanic_weighted[fraction_refor_temperate_oceanic_weighted_abs>5]=np.nan
fraction_defor_temperate_oceanic_weighted[fraction_defor_temperate_oceanic_weighted_abs>5]=np.nan
fraction_refor_temperate_oceanic_weighted_abs[fraction_refor_temperate_oceanic_weighted_abs>5]=np.nan
fraction_defor_temperate_oceanic_weighted_abs[fraction_defor_temperate_oceanic_weighted_abs>5]=np.nan

fraction_temperate_oceanic_weighted_mean = np.nanmean(np.concatenate((fraction_refor_temperate_oceanic_weighted,fraction_defor_temperate_oceanic_weighted),axis=0))
fraction_temperate_oceanic_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_temperate_oceanic_weighted_abs,fraction_defor_temperate_oceanic_weighted_abs),axis=0))
fraction_temperate_oceanic_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_refor_temperate_oceanic_weighted,fraction_defor_temperate_oceanic_weighted),axis=0),5)
fraction_temperate_oceanic_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_refor_temperate_oceanic_weighted,fraction_defor_temperate_oceanic_weighted),axis=0),95)



fraction_neg_refor_temperate_oceanic_weighted[fraction_neg_refor_temperate_oceanic_weighted>5]=5
fraction_neg_defor_temperate_oceanic_weighted[fraction_neg_defor_temperate_oceanic_weighted>5]=5
fraction_neg_refor_temperate_oceanic_weighted_mean = np.nanmean(fraction_neg_refor_temperate_oceanic_weighted)
fraction_neg_defor_temperate_oceanic_weighted_mean = np.nanmean(fraction_neg_defor_temperate_oceanic_weighted)


fraction_neg_temperate_oceanic_weighted_mean = np.nanmean(np.concatenate((fraction_neg_refor_temperate_oceanic_weighted,fraction_neg_defor_temperate_oceanic_weighted),axis=0))
fraction_neg_temperate_oceanic_weighted_median = np.nanmedian(np.concatenate((fraction_neg_refor_temperate_oceanic_weighted,fraction_neg_defor_temperate_oceanic_weighted),axis=0))


fraction_temperate_oceanic_weighted_std_abs = np.nanstd(np.concatenate((fraction_refor_temperate_oceanic_weighted_abs,fraction_defor_temperate_oceanic_weighted_abs),axis=0))
fraction_temperate_oceanic_weighted_std = np.nanstd(np.concatenate((fraction_refor_temperate_oceanic_weighted,fraction_defor_temperate_oceanic_weighted),axis=0))



"""
TEMPERATE_CONTINENTAL
"""
###MEAN
refor_temperate_continental = np.where(refor_local*temperate_continental_mask!=0,refor_local*temperate_continental_mask,np.nan)
defor_temperate_continental = np.where(defor_local*temperate_continental_mask!=0,defor_local*temperate_continental_mask,np.nan)

refor_temperate_continental_area = np.nansum(~np.isnan(refor_temperate_continental)*area_grid)
defor_temperate_continental_area = np.nansum(~np.isnan(defor_temperate_continental)*area_grid)

total_temperate_continental_mean = (np.nansum(refor_temperate_continental*area_grid)+np.nansum(defor_temperate_continental*area_grid))/(refor_temperate_continental_area+defor_temperate_continental_area)


#RANGE
refor_temperate_continental_area_mask = np.where(~np.isnan(refor_temperate_continental),area_grid,np.nan)
refor_temperate_continental_weighted = area_grid*refor_temperate_continental/np.nanmean(refor_temperate_continental_area_mask)
defor_temperate_continental_area_mask = np.where(~np.isnan(defor_temperate_continental),area_grid,np.nan)
defor_temperate_continental_weighted = area_grid*defor_temperate_continental/np.nanmean(defor_temperate_continental_area_mask)
temperate_continental_std = np.nanstd(np.concatenate((refor_temperate_continental_weighted,defor_temperate_continental_weighted),axis=0))

###dT
dT_refor_temperate_continental = np.where(refor_dT*temperate_continental_mask!=0,refor_dT*temperate_continental_mask,np.nan)
dT_defor_temperate_continental = np.where(defor_dT*temperate_continental_mask!=0,defor_dT*temperate_continental_mask,np.nan)
dT_refor_temperate_continental_weighted = refor_temperate_continental_area_mask*dT_refor_temperate_continental/np.nanmean(refor_temperate_continental_area_mask)
dT_defor_temperate_continental_weighted = defor_temperate_continental_area_mask*dT_defor_temperate_continental/np.nanmean(defor_temperate_continental_area_mask)

dT_neg_refor_temperate_continental = np.where(dT_refor_temperate_continental<0,dT_refor_temperate_continental,np.nan)
dT_neg_defor_temperate_continental = np.where(dT_defor_temperate_continental<0,dT_defor_temperate_continental,np.nan)
dT_neg_refor_temperate_continental_area_mask = np.where(~np.isnan(dT_neg_refor_temperate_continental),area_grid,np.nan)
dT_neg_defor_temperate_continental_area_mask = np.where(~np.isnan(dT_neg_defor_temperate_continental),area_grid,np.nan)
dT_neg_refor_temperate_continental_weighted = area_grid*dT_neg_refor_temperate_continental/np.nanmean(dT_neg_refor_temperate_continental_area_mask)
dT_neg_defor_temperate_continental_weighted = area_grid*dT_neg_defor_temperate_continental/np.nanmean(dT_neg_defor_temperate_continental_area_mask)


dT_neg_refor_temperate_continental_area_perc = 100*(np.nansum([dT_refor_temperate_continental<=0]*area_grid)/(refor_temperate_continental_area))
dT_neg_defor_temperate_continental_area_perc = 100*(np.nansum([dT_defor_temperate_continental<=0]*area_grid)/(defor_temperate_continental_area))
dT_neg_temperate_continental_area_perc = 100*(np.nansum([dT_refor_temperate_continental<=0]*area_grid)+np.nansum([dT_defor_temperate_continental<=0]*area_grid))/(refor_temperate_continental_area+defor_temperate_continental_area)


dT_temperate_continental_mean = (np.nansum(dT_refor_temperate_continental*area_grid)+np.nansum(dT_defor_temperate_continental*area_grid))/(refor_temperate_continental_area+defor_temperate_continental_area)
dT_temperate_continental_std = np.nanstd(np.concatenate((dT_refor_temperate_continental_weighted,dT_defor_temperate_continental_weighted),axis=0))

###dC
dC_refor_temperate_continental = np.where(refor_dC*temperate_continental_mask!=0,refor_dC*temperate_continental_mask,np.nan)
dC_defor_temperate_continental = np.where(defor_dC*temperate_continental_mask!=0,defor_dC*temperate_continental_mask,np.nan)
dC_refor_temperate_continental_weighted = refor_temperate_continental_area_mask*dC_refor_temperate_continental/np.nanmean(refor_temperate_continental_area_mask)
dC_defor_temperate_continental_weighted = defor_temperate_continental_area_mask*dC_defor_temperate_continental/np.nanmean(defor_temperate_continental_area_mask)

dC_neg_refor_temperate_continental = np.where(dT_refor_temperate_continental<0,dC_refor_temperate_continental,np.nan)
dC_neg_defor_temperate_continental = np.where(dT_defor_temperate_continental<0,dC_defor_temperate_continental,np.nan)
dC_neg_refor_temperate_continental_area_mask = np.where(~np.isnan(dC_neg_refor_temperate_continental),area_grid,np.nan)
dC_neg_defor_temperate_continental_area_mask = np.where(~np.isnan(dC_neg_defor_temperate_continental),area_grid,np.nan)
dC_neg_refor_temperate_continental_weighted = area_grid*dC_neg_refor_temperate_continental/np.nanmean(dC_neg_refor_temperate_continental_area_mask)
dC_neg_defor_temperate_continental_weighted = area_grid*dC_neg_defor_temperate_continental/np.nanmean(dC_neg_defor_temperate_continental_area_mask)


dC_temperate_continental_mean = (np.nansum(dC_refor_temperate_continental*area_grid)+np.nansum(dC_defor_temperate_continental*area_grid))/(refor_temperate_continental_area+defor_temperate_continental_area)


###FRACTION
fraction_refor_temperate_continental = dT_refor_temperate_continental/dC_refor_temperate_continental
fraction_defor_temperate_continental = dT_defor_temperate_continental/dC_defor_temperate_continental
fraction_refor_temperate_continental_weighted = dT_refor_temperate_continental_weighted/dC_refor_temperate_continental_weighted
fraction_defor_temperate_continental_weighted = dT_defor_temperate_continental_weighted/dC_defor_temperate_continental_weighted
fraction_refor_temperate_continental_weighted_abs = np.abs(dT_refor_temperate_continental_weighted)/np.abs(dC_refor_temperate_continental_weighted)
fraction_defor_temperate_continental_weighted_abs = np.abs(dT_defor_temperate_continental_weighted)/np.abs(dC_defor_temperate_continental_weighted)
fraction_neg_refor_temperate_continental_weighted = dT_neg_refor_temperate_continental_weighted/dC_neg_refor_temperate_continental_weighted
fraction_neg_defor_temperate_continental_weighted = dT_neg_defor_temperate_continental_weighted/dC_neg_defor_temperate_continental_weighted

#fraction_temperate_continental_mean = (np.nansum(fraction_refor_temperate_continental*area_grid)+np.nansum(fraction_defor_temperate_continental*area_grid))/(refor_temperate_continental_area+defor_temperate_continental_area)


fraction_temperate_continental_weighted_median = np.nanmedian(np.concatenate((fraction_refor_temperate_continental_weighted,fraction_defor_temperate_continental_weighted),axis=0))
fraction_temperate_continental_weighted_median_abs = np.nanmedian(np.concatenate((fraction_refor_temperate_continental_weighted_abs,fraction_defor_temperate_continental_weighted_abs),axis=0))
fraction_temperate_continental_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_temperate_continental_weighted_abs,fraction_defor_temperate_continental_weighted_abs),axis=0))

#Get rid of outliers
print(np.count_nonzero(np.concatenate((fraction_refor_temperate_continental_weighted_abs,fraction_defor_temperate_continental_weighted_abs),axis=0)>5)/np.count_nonzero(~np.isnan(np.concatenate((fraction_refor_temperate_continental_weighted_abs,fraction_defor_temperate_continental_weighted_abs),axis=0))))
fraction_refor_temperate_continental_weighted[fraction_refor_temperate_continental_weighted_abs>5]=np.nan
fraction_defor_temperate_continental_weighted[fraction_defor_temperate_continental_weighted_abs>5]=np.nan
fraction_refor_temperate_continental_weighted_abs[fraction_refor_temperate_continental_weighted_abs>5]=np.nan
fraction_defor_temperate_continental_weighted_abs[fraction_defor_temperate_continental_weighted_abs>5]=np.nan

fraction_temperate_continental_weighted_mean = np.nanmean(np.concatenate((fraction_refor_temperate_continental_weighted,fraction_defor_temperate_continental_weighted),axis=0))
fraction_temperate_continental_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_temperate_continental_weighted_abs,fraction_defor_temperate_continental_weighted_abs),axis=0))
fraction_temperate_continental_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_refor_temperate_continental_weighted,fraction_defor_temperate_continental_weighted),axis=0),5)
fraction_temperate_continental_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_refor_temperate_continental_weighted,fraction_defor_temperate_continental_weighted),axis=0),95)



fraction_neg_refor_temperate_continental_weighted[fraction_neg_refor_temperate_continental_weighted>5]=5
fraction_neg_defor_temperate_continental_weighted[fraction_neg_defor_temperate_continental_weighted>5]=5
fraction_neg_refor_temperate_continental_weighted_mean = np.nanmean(fraction_neg_refor_temperate_continental_weighted)
fraction_neg_defor_temperate_continental_weighted_mean = np.nanmean(fraction_neg_defor_temperate_continental_weighted)


fraction_neg_temperate_continental_weighted_mean = np.nanmean(np.concatenate((fraction_neg_refor_temperate_continental_weighted,fraction_neg_defor_temperate_continental_weighted),axis=0))
fraction_neg_temperate_continental_weighted_median = np.nanmedian(np.concatenate((fraction_neg_refor_temperate_continental_weighted,fraction_neg_defor_temperate_continental_weighted),axis=0))


fraction_temperate_continental_weighted_std_abs = np.nanstd(np.concatenate((fraction_refor_temperate_continental_weighted_abs,fraction_defor_temperate_continental_weighted_abs),axis=0))
fraction_temperate_continental_weighted_std = np.nanstd(np.concatenate((fraction_refor_temperate_continental_weighted,fraction_defor_temperate_continental_weighted),axis=0))






"""
BOREAL
"""
###MEAN
refor_boreal = np.where(refor_local*boreal_mask!=0,refor_local*boreal_mask,np.nan)
defor_boreal = np.where(defor_local*boreal_mask!=0,defor_local*boreal_mask,np.nan)

refor_boreal_area = np.nansum(~np.isnan(refor_boreal)*area_grid)
defor_boreal_area = np.nansum(~np.isnan(defor_boreal)*area_grid)

total_boreal_mean = (np.nansum(refor_boreal*area_grid)+np.nansum(defor_boreal*area_grid))/(refor_boreal_area+defor_boreal_area)

#RANGE
refor_boreal_area_mask = np.where(~np.isnan(refor_boreal),area_grid,np.nan)
refor_boreal_weighted = area_grid*refor_boreal/np.nanmean(refor_boreal_area_mask)
defor_boreal_area_mask = np.where(~np.isnan(defor_boreal),area_grid,np.nan)
defor_boreal_weighted = area_grid*defor_boreal/np.nanmean(defor_boreal_area_mask)
boreal_std = np.nanstd(np.concatenate((refor_boreal_weighted,defor_boreal_weighted),axis=0))



###dT
dT_refor_boreal = np.where(refor_dT*boreal_mask!=0,refor_dT*boreal_mask,np.nan)
dT_defor_boreal = np.where(defor_dT*boreal_mask!=0,defor_dT*boreal_mask,np.nan)
dT_refor_boreal_weighted = refor_boreal_area_mask*dT_refor_boreal/np.nanmean(refor_boreal_area_mask)
dT_defor_boreal_weighted = defor_boreal_area_mask*dT_defor_boreal/np.nanmean(defor_boreal_area_mask)

dT_neg_refor_boreal = np.where(dT_refor_boreal<0,dT_refor_boreal,np.nan)
dT_neg_defor_boreal = np.where(dT_defor_boreal<0,dT_defor_boreal,np.nan)
dT_neg_refor_boreal_area_mask = np.where(~np.isnan(dT_neg_refor_boreal),area_grid,np.nan)
dT_neg_defor_boreal_area_mask = np.where(~np.isnan(dT_neg_defor_boreal),area_grid,np.nan)
dT_neg_refor_boreal_weighted = area_grid*dT_neg_refor_boreal/np.nanmean(dT_neg_refor_boreal_area_mask)
dT_neg_defor_boreal_weighted = area_grid*dT_neg_defor_boreal/np.nanmean(dT_neg_defor_boreal_area_mask)


dT_neg_refor_boreal_area_perc = 100*(np.nansum([dT_refor_boreal<=0]*area_grid)/(refor_boreal_area))
dT_neg_defor_boreal_area_perc = 100*(np.nansum([dT_defor_boreal<=0]*area_grid)/(defor_boreal_area))
dT_neg_boreal_area_perc = 100*(np.nansum([dT_refor_boreal<=0]*area_grid)+np.nansum([dT_defor_boreal<=0]*area_grid))/(refor_boreal_area+defor_boreal_area)

dT_boreal_mean = (np.nansum(dT_refor_boreal*area_grid)+np.nansum(dT_defor_boreal*area_grid))/(refor_boreal_area+defor_boreal_area)
dT_boreal_std = np.nanstd(np.concatenate((dT_refor_boreal_weighted,dT_defor_boreal_weighted),axis=0))


###dC
dC_refor_boreal = np.where(refor_dC*boreal_mask!=0,refor_dC*boreal_mask,np.nan)
dC_defor_boreal = np.where(defor_dC*boreal_mask!=0,defor_dC*boreal_mask,np.nan)
dC_refor_boreal_weighted = refor_boreal_area_mask*dC_refor_boreal/np.nanmean(refor_boreal_area_mask)
dC_defor_boreal_weighted = defor_boreal_area_mask*dC_defor_boreal/np.nanmean(defor_boreal_area_mask)

dC_boreal_mean = (np.nansum(dC_refor_boreal*area_grid)+np.nansum(dC_defor_boreal*area_grid))/(refor_boreal_area+defor_boreal_area)

dC_neg_refor_boreal = np.where(dT_refor_boreal<0,dC_refor_boreal,np.nan)
dC_neg_defor_boreal = np.where(dT_defor_boreal<0,dC_defor_boreal,np.nan)
dC_neg_refor_boreal_area_mask = np.where(~np.isnan(dC_neg_refor_boreal),area_grid,np.nan)
dC_neg_defor_boreal_area_mask = np.where(~np.isnan(dC_neg_defor_boreal),area_grid,np.nan)
dC_neg_refor_boreal_weighted = area_grid*dC_neg_refor_boreal/np.nanmean(dC_neg_refor_boreal_area_mask)
dC_neg_defor_boreal_weighted = area_grid*dC_neg_defor_boreal/np.nanmean(dC_neg_defor_boreal_area_mask)

###FRACTION
fraction_refor_boreal = dT_refor_boreal/dC_refor_boreal
fraction_defor_boreal = dT_defor_boreal/dC_defor_boreal
fraction_refor_boreal_weighted = dT_refor_boreal_weighted/dC_refor_boreal_weighted
fraction_defor_boreal_weighted = dT_defor_boreal_weighted/dC_defor_boreal_weighted
fraction_refor_boreal_weighted_abs = np.abs(dT_refor_boreal_weighted)/np.abs(dC_refor_boreal_weighted)
fraction_defor_boreal_weighted_abs = np.abs(dT_defor_boreal_weighted)/np.abs(dC_defor_boreal_weighted)
fraction_neg_refor_boreal_weighted = dT_neg_refor_boreal_weighted/dC_neg_refor_boreal_weighted
fraction_neg_defor_boreal_weighted = dT_neg_defor_boreal_weighted/dC_neg_defor_boreal_weighted


#fraction_boreal_mean = (np.nansum(fraction_refor_boreal*area_grid)+np.nansum(fraction_defor_boreal*area_grid))/(refor_boreal_area+defor_boreal_area)

fraction_boreal_weighted_median = np.nanmedian(np.concatenate((fraction_refor_boreal_weighted,fraction_defor_boreal_weighted),axis=0))
fraction_boreal_weighted_median_abs = np.nanmedian(np.concatenate((fraction_refor_boreal_weighted_abs,fraction_defor_boreal_weighted_abs),axis=0))

#Get rid of outliers
print(np.count_nonzero(np.concatenate((fraction_refor_boreal_weighted_abs,fraction_defor_boreal_weighted_abs),axis=0)>5)/np.count_nonzero(~np.isnan(np.concatenate((fraction_refor_boreal_weighted_abs,fraction_defor_boreal_weighted_abs),axis=0))))
fraction_refor_boreal_weighted[fraction_refor_boreal_weighted_abs>5]=np.nan
fraction_defor_boreal_weighted[fraction_defor_boreal_weighted_abs>5]=np.nan
fraction_refor_boreal_weighted_abs[fraction_refor_boreal_weighted_abs>5]=np.nan
fraction_defor_boreal_weighted_abs[fraction_defor_boreal_weighted_abs>5]=np.nan

fraction_boreal_weighted_mean = np.nanmean(np.concatenate((fraction_refor_boreal_weighted,fraction_defor_boreal_weighted),axis=0))
fraction_boreal_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_boreal_weighted_abs,fraction_defor_boreal_weighted_abs),axis=0))
fraction_boreal_weighted_std_abs = np.nanstd(np.concatenate((fraction_refor_boreal_weighted_abs,fraction_defor_boreal_weighted_abs),axis=0))
fraction_boreal_weighted_std = np.nanstd(np.concatenate((fraction_refor_boreal_weighted,fraction_defor_boreal_weighted),axis=0))
fraction_boreal_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_refor_boreal_weighted,fraction_defor_boreal_weighted),axis=0),5)
fraction_boreal_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_refor_boreal_weighted,fraction_defor_boreal_weighted),axis=0),95)


fraction_neg_refor_boreal_weighted[fraction_neg_refor_boreal_weighted>5]=5
fraction_neg_defor_boreal_weighted[fraction_neg_defor_boreal_weighted>5]=5
fraction_neg_refor_boreal_weighted_mean = np.nanmean(fraction_neg_refor_boreal_weighted)
fraction_neg_defor_boreal_weighted_mean = np.nanmean(fraction_neg_defor_boreal_weighted)


fraction_neg_boreal_weighted_mean = np.nanmean(np.concatenate((fraction_neg_refor_boreal_weighted,fraction_neg_defor_boreal_weighted),axis=0))
fraction_neg_boreal_weighted_median = np.nanmedian(np.concatenate((fraction_neg_refor_boreal_weighted,fraction_neg_defor_boreal_weighted),axis=0))


0.001155260997780683
0.005438827838827839
0.006324262169413568
0.004179461880425263
0.0012313657680316488


In [26]:
###Text Annual

textfig1 = f"Our observationally constrained assessment of the combined impact of BGC and BGP effects confirm the strong differences between the suitability of re- and afforestation and forest conservation sites to mitigation of previous model-based studies12–15 (Fig 1.). Low latitudes that sustain tropical rainforests exceed all other regions in terms of mitigation benefit measured by the CO2 equivalent (CO2 eq) with a mean impact per ha of {(total_tropical_mean):.2f} tCO2 eq (standard deviation (σ) +/- {(tropical_std):.2f} tCO2eq/ha). The mitigation benefit of the same area of temperate and boreal forests is on average {(total_tropical_mean/total_temperate_mean):.1f} times (σ: +/-{(tropical_std)/(temperate_std):.1f}) and {(total_tropical_mean/total_boreal_mean):.1f} (σ: +/-{(tropical_std)/(boreal_std):.1f}) times lower than tropical forest. Taking BGP effects into account adds +{(dT_tropical_mean):.2f} tCO2eq/ha (σ: {(dT_tropical_std):.2f} tCO2eq/ha) to the mitigation benefit of tropical regions and +{(dT_temperate_oceanic_mean):.2f}tCO2eq/ha (σ {(dT_temperate_oceanic_std):.2f} tCO2eq/ha) to oceanic temperate forests by cooling the annual local temperature. The CO2 equivalent of the local annual temperature change of temperate continental (μ: {(dT_temperate_continental_mean):.2f}CO2/ha, σ: {(dT_temperate_continental_std):.2f} tCO2eq/ha) and boreal (μ: {(dT_boreal_mean):.2f}CO2/ha, σ: {(dT_boreal_std):.2f} tCO2eq/ha) forests is markedly lower. Planting forests in the latter two regions runs the risk of warming the local climate by BGP effects. The cooling influence of the carbon uptake is opposed by an annual local warming response at {dT_neg_temperate_continental_area_perc:.1f}% and {dT_neg_boreal_area_perc:.1f}% of the total area of continental temperate and boreal regions."
textfig3 = f"The high impact of low latitude tropical forests on climate is predominantly produced by the vast amount of CO2 they store in their biomass. The CO2 equivalent of the annually averaged BGP effects ranges between {fraction_tropical_weighted_perc05*100:.1f}% and {fraction_tropical_weighted_perc95*100:.1f}%  (median {fraction_tropical_weighted_median*100:.1f}%) of the carbon dioxide uptake of tropical forests (Fig 3a). In temperate ({fraction_temperate_weighted_perc05*100:.1f}% - {fraction_temperate_weighted_perc95*100:.1f}%, median {fraction_temperate_weighted_median*100:.1f}%) and boreal ({fraction_boreal_weighted_perc05*100:.1f}% - {fraction_boreal_weighted_perc95*100:.1f}%, median {fraction_boreal_weighted_median*100:.1f}%) regions the BGP effect experiences a higher variance and locally also opposes the cooling effect of the carbon uptake. In the latter two regions the induced temperature change of BGP effects varies markedly with the seasons."

print(textfig1)
print(textfig3)

Our observationally constrained assessment of the combined impact of BGC and BGP effects confirm the strong differences between the suitability of re- and afforestation and forest conservation sites to mitigation of previous model-based studies12–15 (Fig 1.). Low latitudes that sustain tropical rainforests exceed all other regions in terms of mitigation benefit measured by the CO2 equivalent (CO2 eq) with a mean impact per ha of 732.12 tCO2 eq (standard deviation (σ) +/- 167.45 tCO2eq/ha). The mitigation benefit of the same area of temperate and boreal forests is on average 3.6 times (σ: +/-1.4) and 3.7 (σ: +/-2.8) times lower than tropical forest. Taking BGP effects into account adds +51.10 tCO2eq/ha (σ: 21.41 tCO2eq/ha) to the mitigation benefit of tropical regions and +31.80tCO2eq/ha (σ 24.32 tCO2eq/ha) to oceanic temperate forests by cooling the annual local temperature. The CO2 equivalent of the local annual temperature change of temperate continental (μ: 12.02CO2/ha, σ: 17.90 tCO

In [20]:
"""
WINTER
"""

########################################
TCR = "local" #local;global
season = "Annual" #Annual; JJA; DJF
stat = "median" #median;STD
path = "C:/Users/windisch/Desktop/Manuscript_Revised/Final_Revision/BGPvBGC_data" #SPECIFY_DATA_PATH
########################################


"""
ANNUAL
"""
defor_local = np.asarray(Dataset(f"{path}/full_defor_{season}_TCR_{TCR}_{stat}.nc","r")["data"][:])
refor_local = np.asarray(Dataset(f"{path}/full_refor_{season}_TCR_{TCR}_{stat}.nc","r")["data"][:])

defor_dT = np.asarray(Dataset(f"{path}/dT_defor_{season}_TCR_{TCR}_{stat}.nc","r")["data"][:])
refor_dT = np.asarray(Dataset(f"{path}/dT_refor_{season}_TCR_{TCR}_{stat}.nc","r")["data"][:])

defor_dC = np.asarray(Dataset(f"{path}/dC_defor_{season}_TCR_X_{stat}.nc","r")["data"][:])
refor_dC = np.asarray(Dataset(f"{path}/dC_refor_{season}_TCR_X_{stat}.nc","r")["data"][:])

defor_local = np.flip(defor_local,axis=0)
refor_local = np.flip(refor_local,axis=0)
defor_dT = np.flip(defor_dT,axis=0)
refor_dT = np.flip(refor_dT,axis=0)
defor_dC = np.flip(defor_dC,axis=0)
refor_dC = np.flip(refor_dC,axis=0)

###Number Calculation
"""
TROPICAL
"""

##MEAN
refor_tropical = np.where(refor_local*tropical_mask!=0,refor_local*tropical_mask,np.nan)
defor_tropical = np.where(defor_local*tropical_mask!=0,defor_local*tropical_mask,np.nan)

refor_tropical_area = np.nansum(~np.isnan(refor_tropical)*area_grid)
defor_tropical_area = np.nansum(~np.isnan(defor_tropical)*area_grid)


total_tropical_mean = (np.nansum(refor_tropical*area_grid)+np.nansum(defor_tropical*area_grid))/(refor_tropical_area+defor_tropical_area)

#RANGE
refor_tropical_area_mask = np.where(~np.isnan(refor_tropical),area_grid,np.nan)
refor_tropical_weighted = area_grid*refor_tropical/np.nanmean(refor_tropical_area_mask)
defor_tropical_area_mask = np.where(~np.isnan(defor_tropical),area_grid,np.nan)
defor_tropical_weighted = area_grid*defor_tropical/np.nanmean(defor_tropical_area_mask)
tropical_std = np.nanstd(np.concatenate((refor_tropical_weighted,defor_tropical_weighted),axis=0))

###dT
dT_refor_tropical = np.where(refor_dT*tropical_mask!=0,refor_dT*tropical_mask,np.nan)
dT_defor_tropical = np.where(defor_dT*tropical_mask!=0,defor_dT*tropical_mask,np.nan)
dT_refor_tropical_weighted = refor_tropical_area_mask*dT_refor_tropical/np.nanmean(refor_tropical_area_mask)
dT_defor_tropical_weighted = defor_tropical_area_mask*dT_defor_tropical/np.nanmean(defor_tropical_area_mask)

dT_tropical_mean = (np.nansum(dT_refor_tropical*area_grid)+np.nansum(dT_defor_tropical*area_grid))/(refor_tropical_area+defor_tropical_area)
dT_tropical_std = np.nanstd(np.concatenate((dT_refor_tropical_weighted,dT_defor_tropical_weighted),axis=0))

###dC
dC_refor_tropical = np.where(refor_dC*tropical_mask!=0,refor_dC*tropical_mask,np.nan)
dC_defor_tropical = np.where(defor_dC*tropical_mask!=0,defor_dC*tropical_mask,np.nan)
dC_refor_tropical_weighted = refor_tropical_area_mask*dC_refor_tropical/np.nanmean(refor_tropical_area_mask)
dC_defor_tropical_weighted = defor_tropical_area_mask*dC_defor_tropical/np.nanmean(defor_tropical_area_mask)

dC_tropical_mean = (np.nansum(dC_refor_tropical*area_grid)+np.nansum(dC_defor_tropical*area_grid))/(refor_tropical_area+defor_tropical_area)


###FRACTION
fraction_refor_tropical = dT_refor_tropical/dC_refor_tropical
fraction_defor_tropical = dT_defor_tropical/dC_defor_tropical
fraction_refor_tropical_weighted = dT_refor_tropical_weighted/dC_refor_tropical_weighted
fraction_defor_tropical_weighted = dT_defor_tropical_weighted/dC_defor_tropical_weighted

fraction_refor_tropical_weighted_abs = np.abs(dT_refor_tropical_weighted)/np.abs(dC_refor_tropical_weighted)
fraction_defor_tropical_weighted_abs = np.abs(dT_defor_tropical_weighted)/np.abs(dC_defor_tropical_weighted)


fraction_tropical_mean = (np.nansum(fraction_refor_tropical*area_grid)+np.nansum(fraction_defor_tropical*area_grid))/(refor_tropical_area+defor_tropical_area)

fraction_tropical_weighted_median = np.nanmedian(np.concatenate((fraction_refor_tropical_weighted,fraction_defor_tropical_weighted),axis=0))
fraction_tropical_weighted_median_abs = np.nanmedian(np.concatenate((fraction_refor_tropical_weighted_abs,fraction_defor_tropical_weighted_abs),axis=0))
fraction_tropical_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_tropical_weighted_abs,fraction_defor_tropical_weighted_abs),axis=0))


#Get rid of outliers
print(np.count_nonzero(np.concatenate((fraction_refor_tropical_weighted_abs,fraction_defor_tropical_weighted_abs),axis=0)>5)/np.count_nonzero(~np.isnan(np.concatenate((fraction_refor_tropical_weighted_abs,fraction_defor_tropical_weighted_abs),axis=0))))
fraction_refor_tropical_weighted[fraction_refor_tropical_weighted_abs>5]=np.nan
fraction_defor_tropical_weighted[fraction_defor_tropical_weighted_abs>5]=np.nan
fraction_refor_tropical_weighted_abs[fraction_refor_tropical_weighted_abs>5]=np.nan
fraction_defor_tropical_weighted_abs[fraction_defor_tropical_weighted_abs>5]=np.nan

fraction_tropical_weighted_mean = np.nanmean(np.concatenate((fraction_refor_tropical_weighted,fraction_defor_tropical_weighted),axis=0))
fraction_tropical_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_tropical_weighted_abs,fraction_defor_tropical_weighted_abs),axis=0))
fraction_tropical_weighted_std_abs = np.nanstd(np.concatenate((fraction_refor_tropical_weighted_abs,fraction_defor_tropical_weighted_abs),axis=0))
fraction_tropical_weighted_std = np.nanstd(np.concatenate((fraction_refor_tropical_weighted,fraction_defor_tropical_weighted),axis=0))
fraction_tropical_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_refor_tropical_weighted,fraction_defor_tropical_weighted),axis=0),5)
fraction_tropical_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_refor_tropical_weighted,fraction_defor_tropical_weighted),axis=0),95)


"""
TEMPERATE
"""
###MEAN
refor_temperate = np.where(refor_local*temperate_mask!=0,refor_local*temperate_mask,np.nan)
defor_temperate = np.where(defor_local*temperate_mask!=0,defor_local*temperate_mask,np.nan)

refor_temperate_area = np.nansum(~np.isnan(refor_temperate)*area_grid)
defor_temperate_area = np.nansum(~np.isnan(defor_temperate)*area_grid)

total_temperate_mean = (np.nansum(refor_temperate*area_grid)+np.nansum(defor_temperate*area_grid))/(refor_temperate_area+defor_temperate_area)


#RANGE
refor_temperate_area_mask = np.where(~np.isnan(refor_temperate),area_grid,np.nan)
refor_temperate_weighted = area_grid*refor_temperate/np.nanmean(refor_temperate_area_mask)
defor_temperate_area_mask = np.where(~np.isnan(defor_temperate),area_grid,np.nan)
defor_temperate_weighted = area_grid*defor_temperate/np.nanmean(defor_temperate_area_mask)
temperate_std = np.nanstd(np.concatenate((refor_temperate_weighted,defor_temperate_weighted),axis=0))

###dT
dT_refor_temperate = np.where(refor_dT*temperate_mask!=0,refor_dT*temperate_mask,np.nan)
dT_defor_temperate = np.where(defor_dT*temperate_mask!=0,defor_dT*temperate_mask,np.nan)
dT_refor_temperate_weighted = refor_temperate_area_mask*dT_refor_temperate/np.nanmean(refor_temperate_area_mask)
dT_defor_temperate_weighted = defor_temperate_area_mask*dT_defor_temperate/np.nanmean(defor_temperate_area_mask)

dT_neg_refor_temperate = np.where(dT_refor_temperate<0,dT_refor_temperate,np.nan)
dT_neg_defor_temperate = np.where(dT_defor_temperate<0,dT_defor_temperate,np.nan)
dT_neg_refor_temperate_area_mask = np.where(~np.isnan(dT_neg_refor_temperate),area_grid,np.nan)
dT_neg_defor_temperate_area_mask = np.where(~np.isnan(dT_neg_defor_temperate),area_grid,np.nan)
dT_neg_refor_temperate_weighted = area_grid*dT_neg_refor_temperate/np.nanmean(dT_neg_refor_temperate_area_mask)
dT_neg_defor_temperate_weighted = area_grid*dT_neg_defor_temperate/np.nanmean(dT_neg_defor_temperate_area_mask)


dT_neg_refor_temperate_area_perc = 100*(np.nansum([dT_refor_temperate<=0]*area_grid)/(refor_temperate_area))
dT_neg_defor_temperate_area_perc = 100*(np.nansum([dT_defor_temperate<=0]*area_grid)/(defor_temperate_area))
dT_neg_temperate_area_perc = 100*(np.nansum([dT_refor_temperate<=0]*area_grid)+np.nansum([dT_defor_temperate<=0]*area_grid))/(refor_temperate_area+defor_temperate_area)


dT_temperate_mean = (np.nansum(dT_refor_temperate*area_grid)+np.nansum(dT_defor_temperate*area_grid))/(refor_temperate_area+defor_temperate_area)
dT_temperate_std = np.nanstd(np.concatenate((dT_refor_temperate_weighted,dT_defor_temperate_weighted),axis=0))

###dC
dC_refor_temperate = np.where(refor_dC*temperate_mask!=0,refor_dC*temperate_mask,np.nan)
dC_defor_temperate = np.where(defor_dC*temperate_mask!=0,defor_dC*temperate_mask,np.nan)
dC_refor_temperate_weighted = refor_temperate_area_mask*dC_refor_temperate/np.nanmean(refor_temperate_area_mask)
dC_defor_temperate_weighted = defor_temperate_area_mask*dC_defor_temperate/np.nanmean(defor_temperate_area_mask)

dC_neg_refor_temperate = np.where(dT_refor_temperate<0,dC_refor_temperate,np.nan)
dC_neg_defor_temperate = np.where(dT_defor_temperate<0,dC_defor_temperate,np.nan)
dC_neg_refor_temperate_area_mask = np.where(~np.isnan(dC_neg_refor_temperate),area_grid,np.nan)
dC_neg_defor_temperate_area_mask = np.where(~np.isnan(dC_neg_defor_temperate),area_grid,np.nan)
dC_neg_refor_temperate_weighted = area_grid*dC_neg_refor_temperate/np.nanmean(dC_neg_refor_temperate_area_mask)
dC_neg_defor_temperate_weighted = area_grid*dC_neg_defor_temperate/np.nanmean(dC_neg_defor_temperate_area_mask)


dC_temperate_mean = (np.nansum(dC_refor_temperate*area_grid)+np.nansum(dC_defor_temperate*area_grid))/(refor_temperate_area+defor_temperate_area)


###FRACTION
fraction_refor_temperate = dT_refor_temperate/dC_refor_temperate
fraction_defor_temperate = dT_defor_temperate/dC_defor_temperate
fraction_refor_temperate_weighted = dT_refor_temperate_weighted/dC_refor_temperate_weighted
fraction_defor_temperate_weighted = dT_defor_temperate_weighted/dC_defor_temperate_weighted
fraction_refor_temperate_weighted_abs = np.abs(dT_refor_temperate_weighted)/np.abs(dC_refor_temperate_weighted)
fraction_defor_temperate_weighted_abs = np.abs(dT_defor_temperate_weighted)/np.abs(dC_defor_temperate_weighted)
fraction_neg_refor_temperate_weighted = dT_neg_refor_temperate_weighted/dC_neg_refor_temperate_weighted
fraction_neg_defor_temperate_weighted = dT_neg_defor_temperate_weighted/dC_neg_defor_temperate_weighted

fraction_temperate_mean = (np.nansum(fraction_refor_temperate*area_grid)+np.nansum(fraction_defor_temperate*area_grid))/(refor_temperate_area+defor_temperate_area)


fraction_temperate_weighted_median = np.nanmedian(np.concatenate((fraction_refor_temperate_weighted,fraction_defor_temperate_weighted),axis=0))
fraction_temperate_weighted_median_abs = np.nanmedian(np.concatenate((fraction_refor_temperate_weighted_abs,fraction_defor_temperate_weighted_abs),axis=0))
fraction_temperate_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_temperate_weighted_abs,fraction_defor_temperate_weighted_abs),axis=0))


#Get rid of outliers
print(np.count_nonzero(np.concatenate((fraction_refor_temperate_weighted_abs,fraction_defor_temperate_weighted_abs),axis=0)>5)/np.count_nonzero(~np.isnan(np.concatenate((fraction_refor_temperate_weighted_abs,fraction_defor_temperate_weighted_abs),axis=0))))
fraction_refor_temperate_weighted[fraction_refor_temperate_weighted_abs>5]=np.nan
fraction_defor_temperate_weighted[fraction_defor_temperate_weighted_abs>5]=np.nan
fraction_refor_temperate_weighted_abs[fraction_refor_temperate_weighted_abs>5]=np.nan
fraction_defor_temperate_weighted_abs[fraction_defor_temperate_weighted_abs>5]=np.nan

fraction_temperate_weighted_mean = np.nanmean(np.concatenate((fraction_refor_temperate_weighted,fraction_defor_temperate_weighted),axis=0))
fraction_temperate_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_temperate_weighted_abs,fraction_defor_temperate_weighted_abs),axis=0))
fraction_temperate_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_refor_temperate_weighted,fraction_defor_temperate_weighted),axis=0),5)
fraction_temperate_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_refor_temperate_weighted,fraction_defor_temperate_weighted),axis=0),95)

fraction_neg_refor_temperate_weighted[fraction_neg_refor_temperate_weighted<-5]=np.nan
fraction_neg_defor_temperate_weighted[fraction_neg_defor_temperate_weighted<-5]=np.nan
fraction_neg_refor_temperate_weighted[fraction_neg_refor_temperate_weighted>0]=np.nan
fraction_neg_defor_temperate_weighted[fraction_neg_defor_temperate_weighted>0]=np.nan

fraction_neg_refor_temperate_weighted_mean = np.nanmean(fraction_neg_refor_temperate_weighted)
fraction_neg_defor_temperate_weighted_mean = np.nanmean(fraction_neg_defor_temperate_weighted)

fraction_neg_temperate_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_neg_refor_temperate_weighted,fraction_neg_defor_temperate_weighted),axis=0),5)
fraction_neg_temperate_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_neg_refor_temperate_weighted,fraction_neg_defor_temperate_weighted),axis=0),95)
fraction_neg_temperate_weighted_mean = np.nanmean(np.concatenate((fraction_neg_refor_temperate_weighted,fraction_neg_defor_temperate_weighted),axis=0))
fraction_neg_temperate_weighted_median = np.nanmedian(np.concatenate((fraction_neg_refor_temperate_weighted,fraction_neg_defor_temperate_weighted),axis=0))
fraction_neg_temperate_weighted_std = np.nanstd(np.concatenate((fraction_neg_refor_temperate_weighted,fraction_neg_defor_temperate_weighted),axis=0))


fraction_temperate_weighted_std_abs = np.nanstd(np.concatenate((fraction_refor_temperate_weighted_abs,fraction_defor_temperate_weighted_abs),axis=0))



"""
TEMPERATE_OCEANIC
"""
###MEAN
refor_temperate_oceanic = np.where(refor_local*temperate_oceanic_mask!=0,refor_local*temperate_oceanic_mask,np.nan)
defor_temperate_oceanic = np.where(defor_local*temperate_oceanic_mask!=0,defor_local*temperate_oceanic_mask,np.nan)

refor_temperate_oceanic_area = np.nansum(~np.isnan(refor_temperate_oceanic)*area_grid)
defor_temperate_oceanic_area = np.nansum(~np.isnan(defor_temperate_oceanic)*area_grid)

total_temperate_oceanic_mean = (np.nansum(refor_temperate_oceanic*area_grid)+np.nansum(defor_temperate_oceanic*area_grid))/(refor_temperate_oceanic_area+defor_temperate_oceanic_area)


#RANGE
refor_temperate_oceanic_area_mask = np.where(~np.isnan(refor_temperate_oceanic),area_grid,np.nan)
refor_temperate_oceanic_weighted = area_grid*refor_temperate_oceanic/np.nanmean(refor_temperate_oceanic_area_mask)
defor_temperate_oceanic_area_mask = np.where(~np.isnan(defor_temperate_oceanic),area_grid,np.nan)
defor_temperate_oceanic_weighted = area_grid*defor_temperate_oceanic/np.nanmean(defor_temperate_oceanic_area_mask)
temperate_oceanic_std = np.nanstd(np.concatenate((refor_temperate_oceanic_weighted,defor_temperate_oceanic_weighted),axis=0))

###dT
dT_refor_temperate_oceanic = np.where(refor_dT*temperate_oceanic_mask!=0,refor_dT*temperate_oceanic_mask,np.nan)
dT_defor_temperate_oceanic = np.where(defor_dT*temperate_oceanic_mask!=0,defor_dT*temperate_oceanic_mask,np.nan)
dT_refor_temperate_oceanic_weighted = refor_temperate_oceanic_area_mask*dT_refor_temperate_oceanic/np.nanmean(refor_temperate_oceanic_area_mask)
dT_defor_temperate_oceanic_weighted = defor_temperate_oceanic_area_mask*dT_defor_temperate_oceanic/np.nanmean(defor_temperate_oceanic_area_mask)

dT_neg_refor_temperate_oceanic = np.where(dT_refor_temperate_oceanic<0,dT_refor_temperate_oceanic,np.nan)
dT_neg_defor_temperate_oceanic = np.where(dT_defor_temperate_oceanic<0,dT_defor_temperate_oceanic,np.nan)
dT_neg_refor_temperate_oceanic_area_mask = np.where(~np.isnan(dT_neg_refor_temperate_oceanic),area_grid,np.nan)
dT_neg_defor_temperate_oceanic_area_mask = np.where(~np.isnan(dT_neg_defor_temperate_oceanic),area_grid,np.nan)
dT_neg_refor_temperate_oceanic_weighted = area_grid*dT_neg_refor_temperate_oceanic/np.nanmean(dT_neg_refor_temperate_oceanic_area_mask)
dT_neg_defor_temperate_oceanic_weighted = area_grid*dT_neg_defor_temperate_oceanic/np.nanmean(dT_neg_defor_temperate_oceanic_area_mask)


dT_neg_refor_temperate_oceanic_area_perc = 100*(np.nansum([dT_refor_temperate_oceanic<=0]*area_grid)/(refor_temperate_oceanic_area))
dT_neg_defor_temperate_oceanic_area_perc = 100*(np.nansum([dT_defor_temperate_oceanic<=0]*area_grid)/(defor_temperate_oceanic_area))
dT_neg_temperate_oceanic_area_perc = 100*(np.nansum([dT_refor_temperate_oceanic<=0]*area_grid)+np.nansum([dT_defor_temperate_oceanic<=0]*area_grid))/(refor_temperate_oceanic_area+defor_temperate_oceanic_area)


dT_temperate_oceanic_mean = (np.nansum(dT_refor_temperate_oceanic*area_grid)+np.nansum(dT_defor_temperate_oceanic*area_grid))/(refor_temperate_oceanic_area+defor_temperate_oceanic_area)
dT_temperate_oceanic_std = np.nanstd(np.concatenate((dT_refor_temperate_oceanic_weighted,dT_defor_temperate_oceanic_weighted),axis=0))

###dC
dC_refor_temperate_oceanic = np.where(refor_dC*temperate_oceanic_mask!=0,refor_dC*temperate_oceanic_mask,np.nan)
dC_defor_temperate_oceanic = np.where(defor_dC*temperate_oceanic_mask!=0,defor_dC*temperate_oceanic_mask,np.nan)
dC_refor_temperate_oceanic_weighted = refor_temperate_oceanic_area_mask*dC_refor_temperate_oceanic/np.nanmean(refor_temperate_oceanic_area_mask)
dC_defor_temperate_oceanic_weighted = defor_temperate_oceanic_area_mask*dC_defor_temperate_oceanic/np.nanmean(defor_temperate_oceanic_area_mask)

dC_neg_refor_temperate_oceanic = np.where(dT_refor_temperate_oceanic<0,dC_refor_temperate_oceanic,np.nan)
dC_neg_defor_temperate_oceanic = np.where(dT_defor_temperate_oceanic<0,dC_defor_temperate_oceanic,np.nan)
dC_neg_refor_temperate_oceanic_area_mask = np.where(~np.isnan(dC_neg_refor_temperate_oceanic),area_grid,np.nan)
dC_neg_defor_temperate_oceanic_area_mask = np.where(~np.isnan(dC_neg_defor_temperate_oceanic),area_grid,np.nan)
dC_neg_refor_temperate_oceanic_weighted = area_grid*dC_neg_refor_temperate_oceanic/np.nanmean(dC_neg_refor_temperate_oceanic_area_mask)
dC_neg_defor_temperate_oceanic_weighted = area_grid*dC_neg_defor_temperate_oceanic/np.nanmean(dC_neg_defor_temperate_oceanic_area_mask)


dC_temperate_oceanic_mean = (np.nansum(dC_refor_temperate_oceanic*area_grid)+np.nansum(dC_defor_temperate_oceanic*area_grid))/(refor_temperate_oceanic_area+defor_temperate_oceanic_area)


###FRACTION
fraction_refor_temperate_oceanic = dT_refor_temperate_oceanic/dC_refor_temperate_oceanic
fraction_defor_temperate_oceanic = dT_defor_temperate_oceanic/dC_defor_temperate_oceanic
fraction_refor_temperate_oceanic_weighted = dT_refor_temperate_oceanic_weighted/dC_refor_temperate_oceanic_weighted
fraction_defor_temperate_oceanic_weighted = dT_defor_temperate_oceanic_weighted/dC_defor_temperate_oceanic_weighted
fraction_refor_temperate_oceanic_weighted_abs = np.abs(dT_refor_temperate_oceanic_weighted)/np.abs(dC_refor_temperate_oceanic_weighted)
fraction_defor_temperate_oceanic_weighted_abs = np.abs(dT_defor_temperate_oceanic_weighted)/np.abs(dC_defor_temperate_oceanic_weighted)
fraction_neg_refor_temperate_oceanic_weighted = dT_neg_refor_temperate_oceanic_weighted/dC_neg_refor_temperate_oceanic_weighted
fraction_neg_defor_temperate_oceanic_weighted = dT_neg_defor_temperate_oceanic_weighted/dC_neg_defor_temperate_oceanic_weighted

fraction_temperate_oceanic_mean = (np.nansum(fraction_refor_temperate_oceanic*area_grid)+np.nansum(fraction_defor_temperate_oceanic*area_grid))/(refor_temperate_oceanic_area+defor_temperate_oceanic_area)


fraction_temperate_oceanic_weighted_median = np.nanmedian(np.concatenate((fraction_refor_temperate_oceanic_weighted,fraction_defor_temperate_oceanic_weighted),axis=0))
fraction_temperate_oceanic_weighted_median_abs = np.nanmedian(np.concatenate((fraction_refor_temperate_oceanic_weighted_abs,fraction_defor_temperate_oceanic_weighted_abs),axis=0))
fraction_temperate_oceanic_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_temperate_oceanic_weighted_abs,fraction_defor_temperate_oceanic_weighted_abs),axis=0))

#Get rid of outliers
print(np.count_nonzero(np.concatenate((fraction_refor_temperate_oceanic_weighted_abs,fraction_defor_temperate_oceanic_weighted_abs),axis=0)>5)/np.count_nonzero(~np.isnan(np.concatenate((fraction_refor_temperate_oceanic_weighted_abs,fraction_defor_temperate_oceanic_weighted_abs),axis=0))))
fraction_refor_temperate_oceanic_weighted[fraction_refor_temperate_oceanic_weighted_abs>5]=np.nan
fraction_defor_temperate_oceanic_weighted[fraction_defor_temperate_oceanic_weighted_abs>5]=np.nan
fraction_refor_temperate_oceanic_weighted_abs[fraction_refor_temperate_oceanic_weighted_abs>5]=np.nan
fraction_defor_temperate_oceanic_weighted_abs[fraction_defor_temperate_oceanic_weighted_abs>5]=np.nan

fraction_temperate_oceanic_weighted_mean = np.nanmean(np.concatenate((fraction_refor_temperate_oceanic_weighted,fraction_defor_temperate_oceanic_weighted),axis=0))
fraction_temperate_oceanic_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_temperate_oceanic_weighted_abs,fraction_defor_temperate_oceanic_weighted_abs),axis=0))
fraction_temperate_oceanic_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_refor_temperate_oceanic_weighted,fraction_defor_temperate_oceanic_weighted),axis=0),5)
fraction_temperate_oceanic_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_refor_temperate_oceanic_weighted,fraction_defor_temperate_oceanic_weighted),axis=0),95)

fraction_neg_refor_temperate_oceanic_weighted[fraction_neg_refor_temperate_oceanic_weighted<-5]=np.nan
fraction_neg_defor_temperate_oceanic_weighted[fraction_neg_defor_temperate_oceanic_weighted<-5]=np.nan
fraction_neg_refor_temperate_oceanic_weighted[fraction_neg_refor_temperate_oceanic_weighted>0]=np.nan
fraction_neg_defor_temperate_oceanic_weighted[fraction_neg_defor_temperate_oceanic_weighted>0]=np.nan
fraction_neg_refor_temperate_oceanic_weighted_mean = np.nanmean(fraction_neg_refor_temperate_oceanic_weighted)
fraction_neg_defor_temperate_oceanic_weighted_mean = np.nanmean(fraction_neg_defor_temperate_oceanic_weighted)

fraction_neg_temperate_oceanic_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_neg_refor_temperate_oceanic_weighted,fraction_neg_defor_temperate_oceanic_weighted),axis=0),5)
fraction_neg_temperate_oceanic_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_neg_refor_temperate_oceanic_weighted,fraction_neg_defor_temperate_oceanic_weighted),axis=0),95)
fraction_neg_temperate_oceanic_weighted_mean = np.nanmean(np.concatenate((fraction_neg_refor_temperate_oceanic_weighted,fraction_neg_defor_temperate_oceanic_weighted),axis=0))
fraction_neg_temperate_oceanic_weighted_median = np.nanmedian(np.concatenate((fraction_neg_refor_temperate_oceanic_weighted,fraction_neg_defor_temperate_oceanic_weighted),axis=0))


fraction_temperate_oceanic_weighted_std_abs = np.nanstd(np.concatenate((fraction_refor_temperate_oceanic_weighted_abs,fraction_defor_temperate_oceanic_weighted_abs),axis=0))



"""
TEMPERATE_CONTINENTAL
"""
###MEAN
refor_temperate_continental = np.where(refor_local*temperate_continental_mask!=0,refor_local*temperate_continental_mask,np.nan)
defor_temperate_continental = np.where(defor_local*temperate_continental_mask!=0,defor_local*temperate_continental_mask,np.nan)

refor_temperate_continental_area = np.nansum(~np.isnan(refor_temperate_continental)*area_grid)
defor_temperate_continental_area = np.nansum(~np.isnan(defor_temperate_continental)*area_grid)

total_temperate_continental_mean = (np.nansum(refor_temperate_continental*area_grid)+np.nansum(defor_temperate_continental*area_grid))/(refor_temperate_continental_area+defor_temperate_continental_area)


#RANGE
refor_temperate_continental_area_mask = np.where(~np.isnan(refor_temperate_continental),area_grid,np.nan)
refor_temperate_continental_weighted = area_grid*refor_temperate_continental/np.nanmean(refor_temperate_continental_area_mask)
defor_temperate_continental_area_mask = np.where(~np.isnan(defor_temperate_continental),area_grid,np.nan)
defor_temperate_continental_weighted = area_grid*defor_temperate_continental/np.nanmean(defor_temperate_continental_area_mask)
temperate_continental_std = np.nanstd(np.concatenate((refor_temperate_continental_weighted,defor_temperate_continental_weighted),axis=0))

###dT
dT_refor_temperate_continental = np.where(refor_dT*temperate_continental_mask!=0,refor_dT*temperate_continental_mask,np.nan)
dT_defor_temperate_continental = np.where(defor_dT*temperate_continental_mask!=0,defor_dT*temperate_continental_mask,np.nan)
dT_refor_temperate_continental_weighted = refor_temperate_continental_area_mask*dT_refor_temperate_continental/np.nanmean(refor_temperate_continental_area_mask)
dT_defor_temperate_continental_weighted = defor_temperate_continental_area_mask*dT_defor_temperate_continental/np.nanmean(defor_temperate_continental_area_mask)

dT_neg_refor_temperate_continental = np.where(dT_refor_temperate_continental<0,dT_refor_temperate_continental,np.nan)
dT_neg_defor_temperate_continental = np.where(dT_defor_temperate_continental<0,dT_defor_temperate_continental,np.nan)
dT_neg_refor_temperate_continental_area_mask = np.where(~np.isnan(dT_neg_refor_temperate_continental),area_grid,np.nan)
dT_neg_defor_temperate_continental_area_mask = np.where(~np.isnan(dT_neg_defor_temperate_continental),area_grid,np.nan)
dT_neg_refor_temperate_continental_weighted = area_grid*dT_neg_refor_temperate_continental/np.nanmean(dT_neg_refor_temperate_continental_area_mask)
dT_neg_defor_temperate_continental_weighted = area_grid*dT_neg_defor_temperate_continental/np.nanmean(dT_neg_defor_temperate_continental_area_mask)


dT_neg_refor_temperate_continental_area_perc = 100*(np.nansum([dT_refor_temperate_continental<=0]*area_grid)/(refor_temperate_continental_area))
dT_neg_defor_temperate_continental_area_perc = 100*(np.nansum([dT_defor_temperate_continental<=0]*area_grid)/(defor_temperate_continental_area))
dT_neg_temperate_continental_area_perc = 100*(np.nansum([dT_refor_temperate_continental<=0]*area_grid)+np.nansum([dT_defor_temperate_continental<=0]*area_grid))/(refor_temperate_continental_area+defor_temperate_continental_area)


dT_temperate_continental_mean = (np.nansum(dT_refor_temperate_continental*area_grid)+np.nansum(dT_defor_temperate_continental*area_grid))/(refor_temperate_continental_area+defor_temperate_continental_area)
dT_temperate_continental_std = np.nanstd(np.concatenate((dT_refor_temperate_continental_weighted,dT_defor_temperate_continental_weighted),axis=0))

###dC
dC_refor_temperate_continental = np.where(refor_dC*temperate_continental_mask!=0,refor_dC*temperate_continental_mask,np.nan)
dC_defor_temperate_continental = np.where(defor_dC*temperate_continental_mask!=0,defor_dC*temperate_continental_mask,np.nan)
dC_refor_temperate_continental_weighted = refor_temperate_continental_area_mask*dC_refor_temperate_continental/np.nanmean(refor_temperate_continental_area_mask)
dC_defor_temperate_continental_weighted = defor_temperate_continental_area_mask*dC_defor_temperate_continental/np.nanmean(defor_temperate_continental_area_mask)

dC_neg_refor_temperate_continental = np.where(dT_refor_temperate_continental<0,dC_refor_temperate_continental,np.nan)
dC_neg_defor_temperate_continental = np.where(dT_defor_temperate_continental<0,dC_defor_temperate_continental,np.nan)
dC_neg_refor_temperate_continental_area_mask = np.where(~np.isnan(dC_neg_refor_temperate_continental),area_grid,np.nan)
dC_neg_defor_temperate_continental_area_mask = np.where(~np.isnan(dC_neg_defor_temperate_continental),area_grid,np.nan)
dC_neg_refor_temperate_continental_weighted = area_grid*dC_neg_refor_temperate_continental/np.nanmean(dC_neg_refor_temperate_continental_area_mask)
dC_neg_defor_temperate_continental_weighted = area_grid*dC_neg_defor_temperate_continental/np.nanmean(dC_neg_defor_temperate_continental_area_mask)


dC_temperate_continental_mean = (np.nansum(dC_refor_temperate_continental*area_grid)+np.nansum(dC_defor_temperate_continental*area_grid))/(refor_temperate_continental_area+defor_temperate_continental_area)


###FRACTION
fraction_refor_temperate_continental = dT_refor_temperate_continental/dC_refor_temperate_continental
fraction_defor_temperate_continental = dT_defor_temperate_continental/dC_defor_temperate_continental
fraction_refor_temperate_continental_weighted = dT_refor_temperate_continental_weighted/dC_refor_temperate_continental_weighted
fraction_defor_temperate_continental_weighted = dT_defor_temperate_continental_weighted/dC_defor_temperate_continental_weighted
fraction_refor_temperate_continental_weighted_abs = np.abs(dT_refor_temperate_continental_weighted)/np.abs(dC_refor_temperate_continental_weighted)
fraction_defor_temperate_continental_weighted_abs = np.abs(dT_defor_temperate_continental_weighted)/np.abs(dC_defor_temperate_continental_weighted)
fraction_neg_refor_temperate_continental_weighted = dT_neg_refor_temperate_continental_weighted/dC_neg_refor_temperate_continental_weighted
fraction_neg_defor_temperate_continental_weighted = dT_neg_defor_temperate_continental_weighted/dC_neg_defor_temperate_continental_weighted

fraction_temperate_continental_mean = (np.nansum(fraction_refor_temperate_continental*area_grid)+np.nansum(fraction_defor_temperate_continental*area_grid))/(refor_temperate_continental_area+defor_temperate_continental_area)


fraction_temperate_continental_weighted_median = np.nanmedian(np.concatenate((fraction_refor_temperate_continental_weighted,fraction_defor_temperate_continental_weighted),axis=0))
fraction_temperate_continental_weighted_median_abs = np.nanmedian(np.concatenate((fraction_refor_temperate_continental_weighted_abs,fraction_defor_temperate_continental_weighted_abs),axis=0))
fraction_temperate_continental_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_temperate_continental_weighted_abs,fraction_defor_temperate_continental_weighted_abs),axis=0))

#Get rid of outliers
print(np.count_nonzero(np.concatenate((fraction_refor_temperate_continental_weighted_abs,fraction_defor_temperate_continental_weighted_abs),axis=0)>5)/np.count_nonzero(~np.isnan(np.concatenate((fraction_refor_temperate_continental_weighted_abs,fraction_defor_temperate_continental_weighted_abs),axis=0))))
fraction_refor_temperate_continental_weighted[fraction_refor_temperate_continental_weighted_abs>5]=np.nan
fraction_defor_temperate_continental_weighted[fraction_defor_temperate_continental_weighted_abs>5]=np.nan
fraction_refor_temperate_continental_weighted_abs[fraction_refor_temperate_continental_weighted_abs>5]=np.nan
fraction_defor_temperate_continental_weighted_abs[fraction_defor_temperate_continental_weighted_abs>5]=np.nan

fraction_temperate_continental_weighted_mean = np.nanmean(np.concatenate((fraction_refor_temperate_continental_weighted,fraction_defor_temperate_continental_weighted),axis=0))
fraction_temperate_continental_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_temperate_continental_weighted_abs,fraction_defor_temperate_continental_weighted_abs),axis=0))
fraction_temperate_continental_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_refor_temperate_continental_weighted,fraction_defor_temperate_continental_weighted),axis=0),5)
fraction_temperate_continental_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_refor_temperate_continental_weighted,fraction_defor_temperate_continental_weighted),axis=0),95)

fraction_neg_refor_temperate_continental_weighted[fraction_neg_refor_temperate_continental_weighted<-5]=np.nan
fraction_neg_defor_temperate_continental_weighted[fraction_neg_defor_temperate_continental_weighted<-5]=np.nan
fraction_neg_refor_temperate_continental_weighted[fraction_neg_refor_temperate_continental_weighted>0]=np.nan
fraction_neg_defor_temperate_continental_weighted[fraction_neg_defor_temperate_continental_weighted>0]=np.nan
fraction_neg_refor_temperate_continental_weighted_mean = np.nanmean(fraction_neg_refor_temperate_continental_weighted)
fraction_neg_defor_temperate_continental_weighted_mean = np.nanmean(fraction_neg_defor_temperate_continental_weighted)

fraction_neg_temperate_continental_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_neg_refor_temperate_continental_weighted,fraction_neg_defor_temperate_continental_weighted),axis=0),5)
fraction_neg_temperate_continental_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_neg_refor_temperate_continental_weighted,fraction_neg_defor_temperate_continental_weighted),axis=0),95)
fraction_neg_temperate_continental_weighted_mean = np.nanmean(np.concatenate((fraction_neg_refor_temperate_continental_weighted,fraction_neg_defor_temperate_continental_weighted),axis=0))
fraction_neg_temperate_continental_weighted_median = np.nanmedian(np.concatenate((fraction_neg_refor_temperate_continental_weighted,fraction_neg_defor_temperate_continental_weighted),axis=0))


fraction_temperate_continental_weighted_std_abs = np.nanstd(np.concatenate((fraction_refor_temperate_continental_weighted_abs,fraction_defor_temperate_continental_weighted_abs),axis=0))






"""
BOREAL
"""
###MEAN
refor_boreal = np.where(refor_local*boreal_mask!=0,refor_local*boreal_mask,np.nan)
defor_boreal = np.where(defor_local*boreal_mask!=0,defor_local*boreal_mask,np.nan)

refor_boreal_area = np.nansum(~np.isnan(refor_boreal)*area_grid)
defor_boreal_area = np.nansum(~np.isnan(defor_boreal)*area_grid)

total_boreal_mean = (np.nansum(refor_boreal*area_grid)+np.nansum(defor_boreal*area_grid))/(refor_boreal_area+defor_boreal_area)

#RANGE
refor_boreal_area_mask = np.where(~np.isnan(refor_boreal),area_grid,np.nan)
refor_boreal_weighted = area_grid*refor_boreal/np.nanmean(refor_boreal_area_mask)
defor_boreal_area_mask = np.where(~np.isnan(defor_boreal),area_grid,np.nan)
defor_boreal_weighted = area_grid*defor_boreal/np.nanmean(defor_boreal_area_mask)
boreal_std = np.nanstd(np.concatenate((refor_boreal_weighted,defor_boreal_weighted),axis=0))



###dT
dT_refor_boreal = np.where(refor_dT*boreal_mask!=0,refor_dT*boreal_mask,np.nan)
dT_defor_boreal = np.where(defor_dT*boreal_mask!=0,defor_dT*boreal_mask,np.nan)
dT_refor_boreal_weighted = refor_boreal_area_mask*dT_refor_boreal/np.nanmean(refor_boreal_area_mask)
dT_defor_boreal_weighted = defor_boreal_area_mask*dT_defor_boreal/np.nanmean(defor_boreal_area_mask)

dT_neg_refor_boreal = np.where(dT_refor_boreal<0,dT_refor_boreal,np.nan)
dT_neg_defor_boreal = np.where(dT_defor_boreal<0,dT_defor_boreal,np.nan)
dT_neg_refor_boreal_area_mask = np.where(~np.isnan(dT_neg_refor_boreal),area_grid,np.nan)
dT_neg_defor_boreal_area_mask = np.where(~np.isnan(dT_neg_defor_boreal),area_grid,np.nan)
dT_neg_refor_boreal_weighted = area_grid*dT_neg_refor_boreal/np.nanmean(dT_neg_refor_boreal_area_mask)
dT_neg_defor_boreal_weighted = area_grid*dT_neg_defor_boreal/np.nanmean(dT_neg_defor_boreal_area_mask)


dT_neg_refor_boreal_area_perc = 100*(np.nansum([dT_refor_boreal<=0]*area_grid)/(refor_boreal_area))
dT_neg_defor_boreal_area_perc = 100*(np.nansum([dT_defor_boreal<=0]*area_grid)/(defor_boreal_area))
dT_neg_boreal_area_perc = 100*(np.nansum([dT_refor_boreal<=0]*area_grid)+np.nansum([dT_defor_boreal<=0]*area_grid))/(refor_boreal_area+defor_boreal_area)

dT_boreal_mean = (np.nansum(dT_refor_boreal*area_grid)+np.nansum(dT_defor_boreal*area_grid))/(refor_boreal_area+defor_boreal_area)
dT_boreal_std = np.nanstd(np.concatenate((dT_refor_boreal_weighted,dT_defor_boreal_weighted),axis=0))


###dC
dC_refor_boreal = np.where(refor_dC*boreal_mask!=0,refor_dC*boreal_mask,np.nan)
dC_defor_boreal = np.where(defor_dC*boreal_mask!=0,defor_dC*boreal_mask,np.nan)
dC_refor_boreal_weighted = refor_boreal_area_mask*dC_refor_boreal/np.nanmean(refor_boreal_area_mask)
dC_defor_boreal_weighted = defor_boreal_area_mask*dC_defor_boreal/np.nanmean(defor_boreal_area_mask)

dC_boreal_mean = (np.nansum(dC_refor_boreal*area_grid)+np.nansum(dC_defor_boreal*area_grid))/(refor_boreal_area+defor_boreal_area)

dC_neg_refor_boreal = np.where(dT_refor_boreal<0,dC_refor_boreal,np.nan)
dC_neg_defor_boreal = np.where(dT_defor_boreal<0,dC_defor_boreal,np.nan)
dC_neg_refor_boreal_area_mask = np.where(~np.isnan(dC_neg_refor_boreal),area_grid,np.nan)
dC_neg_defor_boreal_area_mask = np.where(~np.isnan(dC_neg_defor_boreal),area_grid,np.nan)
dC_neg_refor_boreal_weighted = area_grid*dC_neg_refor_boreal/np.nanmean(dC_neg_refor_boreal_area_mask)
dC_neg_defor_boreal_weighted = area_grid*dC_neg_defor_boreal/np.nanmean(dC_neg_defor_boreal_area_mask)

###FRACTION
fraction_refor_boreal = dT_refor_boreal/dC_refor_boreal
fraction_defor_boreal = dT_defor_boreal/dC_defor_boreal
fraction_refor_boreal_weighted = dT_refor_boreal_weighted/dC_refor_boreal_weighted
fraction_defor_boreal_weighted = dT_defor_boreal_weighted/dC_defor_boreal_weighted
fraction_refor_boreal_weighted_abs = np.abs(dT_refor_boreal_weighted)/np.abs(dC_refor_boreal_weighted)
fraction_defor_boreal_weighted_abs = np.abs(dT_defor_boreal_weighted)/np.abs(dC_defor_boreal_weighted)
fraction_neg_refor_boreal_weighted = dT_neg_refor_boreal_weighted/dC_neg_refor_boreal_weighted
fraction_neg_defor_boreal_weighted = dT_neg_defor_boreal_weighted/dC_neg_defor_boreal_weighted


fraction_boreal_mean = (np.nansum(fraction_refor_boreal*area_grid)+np.nansum(fraction_defor_boreal*area_grid))/(refor_boreal_area+defor_boreal_area)

fraction_boreal_weighted_median = np.nanmedian(np.concatenate((fraction_refor_boreal_weighted,fraction_defor_boreal_weighted),axis=0))
fraction_boreal_weighted_median_abs = np.nanmedian(np.concatenate((fraction_refor_boreal_weighted_abs,fraction_defor_boreal_weighted_abs),axis=0))

#Get rid of outliers
print(np.count_nonzero(np.concatenate((fraction_refor_boreal_weighted_abs,fraction_defor_boreal_weighted_abs),axis=0)>5)/np.count_nonzero(~np.isnan(np.concatenate((fraction_refor_boreal_weighted_abs,fraction_defor_boreal_weighted_abs),axis=0))))
fraction_refor_boreal_weighted[fraction_refor_boreal_weighted_abs>5]=np.nan
fraction_defor_boreal_weighted[fraction_defor_boreal_weighted_abs>5]=np.nan
fraction_refor_boreal_weighted_abs[fraction_refor_boreal_weighted_abs>5]=np.nan
fraction_defor_boreal_weighted_abs[fraction_defor_boreal_weighted_abs>5]=np.nan

fraction_boreal_weighted_mean = np.nanmean(np.concatenate((fraction_refor_boreal_weighted,fraction_defor_boreal_weighted),axis=0))
fraction_boreal_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_boreal_weighted_abs,fraction_defor_boreal_weighted_abs),axis=0))
fraction_boreal_weighted_std_abs = np.nanstd(np.concatenate((fraction_refor_boreal_weighted_abs,fraction_defor_boreal_weighted_abs),axis=0))
fraction_boreal_weighted_std = np.nanstd(np.concatenate((fraction_refor_boreal_weighted,fraction_defor_boreal_weighted),axis=0))
fraction_boreal_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_refor_boreal_weighted,fraction_defor_boreal_weighted),axis=0),5)
fraction_boreal_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_refor_boreal_weighted,fraction_defor_boreal_weighted),axis=0),95)


fraction_neg_refor_boreal_weighted[fraction_neg_refor_boreal_weighted<-5]=np.nan
fraction_neg_defor_boreal_weighted[fraction_neg_defor_boreal_weighted<-5]=np.nan
fraction_neg_refor_boreal_weighted[fraction_neg_refor_boreal_weighted>0]=np.nan
fraction_neg_defor_boreal_weighted[fraction_neg_defor_boreal_weighted>0]=np.nan
fraction_neg_refor_boreal_weighted_mean = np.nanmean(fraction_neg_refor_boreal_weighted)
fraction_neg_defor_boreal_weighted_mean = np.nanmean(fraction_neg_defor_boreal_weighted)

fraction_neg_boreal_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_neg_refor_boreal_weighted,fraction_neg_defor_boreal_weighted),axis=0),5)
fraction_neg_boreal_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_neg_refor_boreal_weighted,fraction_neg_defor_boreal_weighted),axis=0),95)
fraction_neg_boreal_weighted_mean = np.nanmean(np.concatenate((fraction_neg_refor_boreal_weighted,fraction_neg_defor_boreal_weighted),axis=0))
fraction_neg_boreal_weighted_median = np.nanmedian(np.concatenate((fraction_neg_refor_boreal_weighted,fraction_neg_defor_boreal_weighted),axis=0))


fraction_neg_boreal_weighted_std = np.nanstd(np.concatenate((fraction_neg_refor_boreal_weighted,fraction_neg_defor_boreal_weighted),axis=0))




0.0011861306886445204
0.009707657156248169
0.0032252062271289463
0.008102595934292491
0.0033618899273104882


In [21]:
###Text Winter
textfig1winter = f"This warming response is even more pronounced in boreal winter where {dT_neg_temperate_continental_area_perc:.1f}% and {dT_neg_boreal_area_perc:.1f}% of the continental temperate and boreal area experience a reduction of their mitigation potential when considering BGP effects."
textfig3winter = f"The cooling influence during summer is reversed in winter. The CO2 eq of the BGP effects of forests during the winter lie between {fraction_temperate_weighted_perc05*100:.1f}% and {fraction_temperate_weighted_perc95*100:.1f}% (median {fraction_temperate_weighted_median*100:.1f}%) of the CO2 uptake at temperate sites and {fraction_boreal_weighted_perc05*100:.1f}% to {fraction_boreal_weighted_perc95*100:.1f}% (median {fraction_boreal_weighted_median*100:.1f}%) of boreal ones. Forest areas that exert a BGP warming see the cooling benefit of the BGC effect reduced by {fraction_neg_temperate_weighted_perc05*100:.1f}% to {fraction_neg_temperate_weighted_perc95*100:.1f}% (median {fraction_neg_temperate_weighted_median*100:.1f}%) in temperate and {fraction_neg_boreal_weighted_perc05*100:.1f}% to {fraction_neg_boreal_weighted_perc95*100:.1f}% (median {fraction_neg_boreal_weighted_median*100:.1f}%) in boreal regions during winter."

print(textfig1winter)
print(textfig3winter)

This warming response is even more pronounced in boreal winter where 73.7% and 68.2% of the continental temperate and boreal area experience a reduction of their mitigation potential when considering BGP effects.
The cooling influence during summer is reversed in winter. The CO2 eq of the BGP effects of forests during the winter lie between -57.2% and 24.4% (median -10.6%) of the CO2 uptake at temperate sites and -32.6% to 23.7% (median -4.1%) of boreal ones. Forest areas that exert a BGP warming see the cooling benefit of the BGC effect reduced by -63.7% to -1.8% (median -18.5%) in temperate and -36.7% to -0.7% (median -10.3%) in boreal regions during winter.


In [22]:
"""
SUMMER
"""

########################################
TCR = "local" #local;global
season = "Annual" #Annual; JJA; DJF
stat = "median" #median;STD
path = "C:/Users/windisch/Desktop/Manuscript_Revised/Final_Revision/BGPvBGC_data" #SPECIFY_DATA_PATH
########################################


"""
ANNUAL
"""
defor_local = np.asarray(Dataset(f"{path}/full_defor_{season}_TCR_{TCR}_{stat}.nc","r")["data"][:])
refor_local = np.asarray(Dataset(f"{path}/full_refor_{season}_TCR_{TCR}_{stat}.nc","r")["data"][:])

defor_dT = np.asarray(Dataset(f"{path}/dT_defor_{season}_TCR_{TCR}_{stat}.nc","r")["data"][:])
refor_dT = np.asarray(Dataset(f"{path}/dT_refor_{season}_TCR_{TCR}_{stat}.nc","r")["data"][:])

defor_dC = np.asarray(Dataset(f"{path}/dC_defor_{season}_TCR_X_{stat}.nc","r")["data"][:])
refor_dC = np.asarray(Dataset(f"{path}/dC_refor_{season}_TCR_X_{stat}.nc","r")["data"][:])

defor_local = np.flip(defor_local,axis=0)
refor_local = np.flip(refor_local,axis=0)
defor_dT = np.flip(defor_dT,axis=0)
refor_dT = np.flip(refor_dT,axis=0)
defor_dC = np.flip(defor_dC,axis=0)
refor_dC = np.flip(refor_dC,axis=0)

###Number Calculation
"""
TROPICAL
"""

##MEAN
refor_tropical = np.where(refor_local*tropical_mask!=0,refor_local*tropical_mask,np.nan)
defor_tropical = np.where(defor_local*tropical_mask!=0,defor_local*tropical_mask,np.nan)

refor_tropical_area = np.nansum(~np.isnan(refor_tropical)*area_grid)
defor_tropical_area = np.nansum(~np.isnan(defor_tropical)*area_grid)


total_tropical_mean = (np.nansum(refor_tropical*area_grid)+np.nansum(defor_tropical*area_grid))/(refor_tropical_area+defor_tropical_area)

#RANGE
refor_tropical_area_mask = np.where(~np.isnan(refor_tropical),area_grid,np.nan)
refor_tropical_weighted = area_grid*refor_tropical/np.nanmean(refor_tropical_area_mask)
defor_tropical_area_mask = np.where(~np.isnan(defor_tropical),area_grid,np.nan)
defor_tropical_weighted = area_grid*defor_tropical/np.nanmean(defor_tropical_area_mask)
tropical_std = np.nanstd(np.concatenate((refor_tropical_weighted,defor_tropical_weighted),axis=0))

###dT
dT_refor_tropical = np.where(refor_dT*tropical_mask!=0,refor_dT*tropical_mask,np.nan)
dT_defor_tropical = np.where(defor_dT*tropical_mask!=0,defor_dT*tropical_mask,np.nan)
dT_refor_tropical_weighted = refor_tropical_area_mask*dT_refor_tropical/np.nanmean(refor_tropical_area_mask)
dT_defor_tropical_weighted = defor_tropical_area_mask*dT_defor_tropical/np.nanmean(defor_tropical_area_mask)

dT_tropical_mean = (np.nansum(dT_refor_tropical*area_grid)+np.nansum(dT_defor_tropical*area_grid))/(refor_tropical_area+defor_tropical_area)
dT_tropical_std = np.nanstd(np.concatenate((dT_refor_tropical_weighted,dT_defor_tropical_weighted),axis=0))

###dC
dC_refor_tropical = np.where(refor_dC*tropical_mask!=0,refor_dC*tropical_mask,np.nan)
dC_defor_tropical = np.where(defor_dC*tropical_mask!=0,defor_dC*tropical_mask,np.nan)
dC_refor_tropical_weighted = refor_tropical_area_mask*dC_refor_tropical/np.nanmean(refor_tropical_area_mask)
dC_defor_tropical_weighted = defor_tropical_area_mask*dC_defor_tropical/np.nanmean(defor_tropical_area_mask)

dC_tropical_mean = (np.nansum(dC_refor_tropical*area_grid)+np.nansum(dC_defor_tropical*area_grid))/(refor_tropical_area+defor_tropical_area)


###FRACTION
fraction_refor_tropical = dT_refor_tropical/dC_refor_tropical
fraction_defor_tropical = dT_defor_tropical/dC_defor_tropical
fraction_refor_tropical_weighted = dT_refor_tropical_weighted/dC_refor_tropical_weighted
fraction_defor_tropical_weighted = dT_defor_tropical_weighted/dC_defor_tropical_weighted

fraction_refor_tropical_weighted_abs = np.abs(dT_refor_tropical_weighted)/np.abs(dC_refor_tropical_weighted)
fraction_defor_tropical_weighted_abs = np.abs(dT_defor_tropical_weighted)/np.abs(dC_defor_tropical_weighted)


fraction_tropical_mean = (np.nansum(fraction_refor_tropical*area_grid)+np.nansum(fraction_defor_tropical*area_grid))/(refor_tropical_area+defor_tropical_area)

fraction_tropical_weighted_median = np.nanmedian(np.concatenate((fraction_refor_tropical_weighted,fraction_defor_tropical_weighted),axis=0))
fraction_tropical_weighted_median_abs = np.nanmedian(np.concatenate((fraction_refor_tropical_weighted_abs,fraction_defor_tropical_weighted_abs),axis=0))
fraction_tropical_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_tropical_weighted_abs,fraction_defor_tropical_weighted_abs),axis=0))


#Get rid of outliers
print(np.count_nonzero(np.concatenate((fraction_refor_tropical_weighted_abs,fraction_defor_tropical_weighted_abs),axis=0)>5)/np.count_nonzero(~np.isnan(np.concatenate((fraction_refor_tropical_weighted_abs,fraction_defor_tropical_weighted_abs),axis=0))))
fraction_refor_tropical_weighted[fraction_refor_tropical_weighted_abs>5]=np.nan
fraction_defor_tropical_weighted[fraction_defor_tropical_weighted_abs>5]=np.nan
fraction_refor_tropical_weighted_abs[fraction_refor_tropical_weighted_abs>5]=np.nan
fraction_defor_tropical_weighted_abs[fraction_defor_tropical_weighted_abs>5]=np.nan

fraction_tropical_weighted_mean = np.nanmean(np.concatenate((fraction_refor_tropical_weighted,fraction_defor_tropical_weighted),axis=0))
fraction_tropical_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_tropical_weighted_abs,fraction_defor_tropical_weighted_abs),axis=0))
fraction_tropical_weighted_std_abs = np.nanstd(np.concatenate((fraction_refor_tropical_weighted_abs,fraction_defor_tropical_weighted_abs),axis=0))
fraction_tropical_weighted_std = np.nanstd(np.concatenate((fraction_refor_tropical_weighted,fraction_defor_tropical_weighted),axis=0))
fraction_tropical_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_refor_tropical_weighted,fraction_defor_tropical_weighted),axis=0),5)
fraction_tropical_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_refor_tropical_weighted,fraction_defor_tropical_weighted),axis=0),95)


"""
TEMPERATE
"""
###MEAN
refor_temperate = np.where(refor_local*temperate_mask!=0,refor_local*temperate_mask,np.nan)
defor_temperate = np.where(defor_local*temperate_mask!=0,defor_local*temperate_mask,np.nan)

refor_temperate_area = np.nansum(~np.isnan(refor_temperate)*area_grid)
defor_temperate_area = np.nansum(~np.isnan(defor_temperate)*area_grid)

total_temperate_mean = (np.nansum(refor_temperate*area_grid)+np.nansum(defor_temperate*area_grid))/(refor_temperate_area+defor_temperate_area)


#RANGE
refor_temperate_area_mask = np.where(~np.isnan(refor_temperate),area_grid,np.nan)
refor_temperate_weighted = area_grid*refor_temperate/np.nanmean(refor_temperate_area_mask)
defor_temperate_area_mask = np.where(~np.isnan(defor_temperate),area_grid,np.nan)
defor_temperate_weighted = area_grid*defor_temperate/np.nanmean(defor_temperate_area_mask)
temperate_std = np.nanstd(np.concatenate((refor_temperate_weighted,defor_temperate_weighted),axis=0))

###dT
dT_refor_temperate = np.where(refor_dT*temperate_mask!=0,refor_dT*temperate_mask,np.nan)
dT_defor_temperate = np.where(defor_dT*temperate_mask!=0,defor_dT*temperate_mask,np.nan)
dT_refor_temperate_weighted = refor_temperate_area_mask*dT_refor_temperate/np.nanmean(refor_temperate_area_mask)
dT_defor_temperate_weighted = defor_temperate_area_mask*dT_defor_temperate/np.nanmean(defor_temperate_area_mask)

dT_neg_refor_temperate = np.where(dT_refor_temperate<0,dT_refor_temperate,np.nan)
dT_neg_defor_temperate = np.where(dT_defor_temperate<0,dT_defor_temperate,np.nan)
dT_neg_refor_temperate_area_mask = np.where(~np.isnan(dT_neg_refor_temperate),area_grid,np.nan)
dT_neg_defor_temperate_area_mask = np.where(~np.isnan(dT_neg_defor_temperate),area_grid,np.nan)
dT_neg_refor_temperate_weighted = area_grid*dT_neg_refor_temperate/np.nanmean(dT_neg_refor_temperate_area_mask)
dT_neg_defor_temperate_weighted = area_grid*dT_neg_defor_temperate/np.nanmean(dT_neg_defor_temperate_area_mask)


dT_neg_refor_temperate_area_perc = 100*(np.nansum([dT_refor_temperate<=0]*area_grid)/(refor_temperate_area))
dT_neg_defor_temperate_area_perc = 100*(np.nansum([dT_defor_temperate<=0]*area_grid)/(defor_temperate_area))
dT_neg_temperate_area_perc = 100*(np.nansum([dT_refor_temperate<=0]*area_grid)+np.nansum([dT_defor_temperate<=0]*area_grid))/(refor_temperate_area+defor_temperate_area)


dT_temperate_mean = (np.nansum(dT_refor_temperate*area_grid)+np.nansum(dT_defor_temperate*area_grid))/(refor_temperate_area+defor_temperate_area)
dT_temperate_std = np.nanstd(np.concatenate((dT_refor_temperate_weighted,dT_defor_temperate_weighted),axis=0))

###dC
dC_refor_temperate = np.where(refor_dC*temperate_mask!=0,refor_dC*temperate_mask,np.nan)
dC_defor_temperate = np.where(defor_dC*temperate_mask!=0,defor_dC*temperate_mask,np.nan)
dC_refor_temperate_weighted = refor_temperate_area_mask*dC_refor_temperate/np.nanmean(refor_temperate_area_mask)
dC_defor_temperate_weighted = defor_temperate_area_mask*dC_defor_temperate/np.nanmean(defor_temperate_area_mask)

dC_neg_refor_temperate = np.where(dT_refor_temperate<0,dC_refor_temperate,np.nan)
dC_neg_defor_temperate = np.where(dT_defor_temperate<0,dC_defor_temperate,np.nan)
dC_neg_refor_temperate_area_mask = np.where(~np.isnan(dC_neg_refor_temperate),area_grid,np.nan)
dC_neg_defor_temperate_area_mask = np.where(~np.isnan(dC_neg_defor_temperate),area_grid,np.nan)
dC_neg_refor_temperate_weighted = area_grid*dC_neg_refor_temperate/np.nanmean(dC_neg_refor_temperate_area_mask)
dC_neg_defor_temperate_weighted = area_grid*dC_neg_defor_temperate/np.nanmean(dC_neg_defor_temperate_area_mask)


dC_temperate_mean = (np.nansum(dC_refor_temperate*area_grid)+np.nansum(dC_defor_temperate*area_grid))/(refor_temperate_area+defor_temperate_area)


###FRACTION
fraction_refor_temperate = dT_refor_temperate/dC_refor_temperate
fraction_defor_temperate = dT_defor_temperate/dC_defor_temperate
fraction_refor_temperate_weighted = dT_refor_temperate_weighted/dC_refor_temperate_weighted
fraction_defor_temperate_weighted = dT_defor_temperate_weighted/dC_defor_temperate_weighted
fraction_refor_temperate_weighted_abs = np.abs(dT_refor_temperate_weighted)/np.abs(dC_refor_temperate_weighted)
fraction_defor_temperate_weighted_abs = np.abs(dT_defor_temperate_weighted)/np.abs(dC_defor_temperate_weighted)
fraction_neg_refor_temperate_weighted = dT_neg_refor_temperate_weighted/dC_neg_refor_temperate_weighted
fraction_neg_defor_temperate_weighted = dT_neg_defor_temperate_weighted/dC_neg_defor_temperate_weighted

fraction_temperate_mean = (np.nansum(fraction_refor_temperate*area_grid)+np.nansum(fraction_defor_temperate*area_grid))/(refor_temperate_area+defor_temperate_area)


fraction_temperate_weighted_median = np.nanmedian(np.concatenate((fraction_refor_temperate_weighted,fraction_defor_temperate_weighted),axis=0))
fraction_temperate_weighted_median_abs = np.nanmedian(np.concatenate((fraction_refor_temperate_weighted_abs,fraction_defor_temperate_weighted_abs),axis=0))
fraction_temperate_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_temperate_weighted_abs,fraction_defor_temperate_weighted_abs),axis=0))


#Get rid of outliers
print(np.count_nonzero(np.concatenate((fraction_refor_temperate_weighted_abs,fraction_defor_temperate_weighted_abs),axis=0)>5)/np.count_nonzero(~np.isnan(np.concatenate((fraction_refor_temperate_weighted_abs,fraction_defor_temperate_weighted_abs),axis=0))))
fraction_refor_temperate_weighted[fraction_refor_temperate_weighted_abs>5]=np.nan
fraction_defor_temperate_weighted[fraction_defor_temperate_weighted_abs>5]=np.nan
fraction_refor_temperate_weighted_abs[fraction_refor_temperate_weighted_abs>5]=np.nan
fraction_defor_temperate_weighted_abs[fraction_defor_temperate_weighted_abs>5]=np.nan

fraction_temperate_weighted_mean = np.nanmean(np.concatenate((fraction_refor_temperate_weighted,fraction_defor_temperate_weighted),axis=0))
fraction_temperate_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_temperate_weighted_abs,fraction_defor_temperate_weighted_abs),axis=0))
fraction_temperate_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_refor_temperate_weighted,fraction_defor_temperate_weighted),axis=0),5)
fraction_temperate_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_refor_temperate_weighted,fraction_defor_temperate_weighted),axis=0),95)

fraction_neg_refor_temperate_weighted[fraction_neg_refor_temperate_weighted<-5]=np.nan
fraction_neg_defor_temperate_weighted[fraction_neg_defor_temperate_weighted<-5]=np.nan
fraction_neg_refor_temperate_weighted[fraction_neg_refor_temperate_weighted>0]=np.nan
fraction_neg_defor_temperate_weighted[fraction_neg_defor_temperate_weighted>0]=np.nan

fraction_neg_refor_temperate_weighted_mean = np.nanmean(fraction_neg_refor_temperate_weighted)
fraction_neg_defor_temperate_weighted_mean = np.nanmean(fraction_neg_defor_temperate_weighted)

fraction_neg_temperate_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_neg_refor_temperate_weighted,fraction_neg_defor_temperate_weighted),axis=0),5)
fraction_neg_temperate_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_neg_refor_temperate_weighted,fraction_neg_defor_temperate_weighted),axis=0),95)
fraction_neg_temperate_weighted_mean = np.nanmean(np.concatenate((fraction_neg_refor_temperate_weighted,fraction_neg_defor_temperate_weighted),axis=0))
fraction_neg_temperate_weighted_median = np.nanmedian(np.concatenate((fraction_neg_refor_temperate_weighted,fraction_neg_defor_temperate_weighted),axis=0))
fraction_neg_temperate_weighted_std = np.nanstd(np.concatenate((fraction_neg_refor_temperate_weighted,fraction_neg_defor_temperate_weighted),axis=0))


fraction_temperate_weighted_std_abs = np.nanstd(np.concatenate((fraction_refor_temperate_weighted_abs,fraction_defor_temperate_weighted_abs),axis=0))



"""
TEMPERATE_OCEANIC
"""
###MEAN
refor_temperate_oceanic = np.where(refor_local*temperate_oceanic_mask!=0,refor_local*temperate_oceanic_mask,np.nan)
defor_temperate_oceanic = np.where(defor_local*temperate_oceanic_mask!=0,defor_local*temperate_oceanic_mask,np.nan)

refor_temperate_oceanic_area = np.nansum(~np.isnan(refor_temperate_oceanic)*area_grid)
defor_temperate_oceanic_area = np.nansum(~np.isnan(defor_temperate_oceanic)*area_grid)

total_temperate_oceanic_mean = (np.nansum(refor_temperate_oceanic*area_grid)+np.nansum(defor_temperate_oceanic*area_grid))/(refor_temperate_oceanic_area+defor_temperate_oceanic_area)


#RANGE
refor_temperate_oceanic_area_mask = np.where(~np.isnan(refor_temperate_oceanic),area_grid,np.nan)
refor_temperate_oceanic_weighted = area_grid*refor_temperate_oceanic/np.nanmean(refor_temperate_oceanic_area_mask)
defor_temperate_oceanic_area_mask = np.where(~np.isnan(defor_temperate_oceanic),area_grid,np.nan)
defor_temperate_oceanic_weighted = area_grid*defor_temperate_oceanic/np.nanmean(defor_temperate_oceanic_area_mask)
temperate_oceanic_std = np.nanstd(np.concatenate((refor_temperate_oceanic_weighted,defor_temperate_oceanic_weighted),axis=0))

###dT
dT_refor_temperate_oceanic = np.where(refor_dT*temperate_oceanic_mask!=0,refor_dT*temperate_oceanic_mask,np.nan)
dT_defor_temperate_oceanic = np.where(defor_dT*temperate_oceanic_mask!=0,defor_dT*temperate_oceanic_mask,np.nan)
dT_refor_temperate_oceanic_weighted = refor_temperate_oceanic_area_mask*dT_refor_temperate_oceanic/np.nanmean(refor_temperate_oceanic_area_mask)
dT_defor_temperate_oceanic_weighted = defor_temperate_oceanic_area_mask*dT_defor_temperate_oceanic/np.nanmean(defor_temperate_oceanic_area_mask)

dT_neg_refor_temperate_oceanic = np.where(dT_refor_temperate_oceanic<0,dT_refor_temperate_oceanic,np.nan)
dT_neg_defor_temperate_oceanic = np.where(dT_defor_temperate_oceanic<0,dT_defor_temperate_oceanic,np.nan)
dT_neg_refor_temperate_oceanic_area_mask = np.where(~np.isnan(dT_neg_refor_temperate_oceanic),area_grid,np.nan)
dT_neg_defor_temperate_oceanic_area_mask = np.where(~np.isnan(dT_neg_defor_temperate_oceanic),area_grid,np.nan)
dT_neg_refor_temperate_oceanic_weighted = area_grid*dT_neg_refor_temperate_oceanic/np.nanmean(dT_neg_refor_temperate_oceanic_area_mask)
dT_neg_defor_temperate_oceanic_weighted = area_grid*dT_neg_defor_temperate_oceanic/np.nanmean(dT_neg_defor_temperate_oceanic_area_mask)


dT_neg_refor_temperate_oceanic_area_perc = 100*(np.nansum([dT_refor_temperate_oceanic<=0]*area_grid)/(refor_temperate_oceanic_area))
dT_neg_defor_temperate_oceanic_area_perc = 100*(np.nansum([dT_defor_temperate_oceanic<=0]*area_grid)/(defor_temperate_oceanic_area))
dT_neg_temperate_oceanic_area_perc = 100*(np.nansum([dT_refor_temperate_oceanic<=0]*area_grid)+np.nansum([dT_defor_temperate_oceanic<=0]*area_grid))/(refor_temperate_oceanic_area+defor_temperate_oceanic_area)


dT_temperate_oceanic_mean = (np.nansum(dT_refor_temperate_oceanic*area_grid)+np.nansum(dT_defor_temperate_oceanic*area_grid))/(refor_temperate_oceanic_area+defor_temperate_oceanic_area)
dT_temperate_oceanic_std = np.nanstd(np.concatenate((dT_refor_temperate_oceanic_weighted,dT_defor_temperate_oceanic_weighted),axis=0))

###dC
dC_refor_temperate_oceanic = np.where(refor_dC*temperate_oceanic_mask!=0,refor_dC*temperate_oceanic_mask,np.nan)
dC_defor_temperate_oceanic = np.where(defor_dC*temperate_oceanic_mask!=0,defor_dC*temperate_oceanic_mask,np.nan)
dC_refor_temperate_oceanic_weighted = refor_temperate_oceanic_area_mask*dC_refor_temperate_oceanic/np.nanmean(refor_temperate_oceanic_area_mask)
dC_defor_temperate_oceanic_weighted = defor_temperate_oceanic_area_mask*dC_defor_temperate_oceanic/np.nanmean(defor_temperate_oceanic_area_mask)

dC_neg_refor_temperate_oceanic = np.where(dT_refor_temperate_oceanic<0,dC_refor_temperate_oceanic,np.nan)
dC_neg_defor_temperate_oceanic = np.where(dT_defor_temperate_oceanic<0,dC_defor_temperate_oceanic,np.nan)
dC_neg_refor_temperate_oceanic_area_mask = np.where(~np.isnan(dC_neg_refor_temperate_oceanic),area_grid,np.nan)
dC_neg_defor_temperate_oceanic_area_mask = np.where(~np.isnan(dC_neg_defor_temperate_oceanic),area_grid,np.nan)
dC_neg_refor_temperate_oceanic_weighted = area_grid*dC_neg_refor_temperate_oceanic/np.nanmean(dC_neg_refor_temperate_oceanic_area_mask)
dC_neg_defor_temperate_oceanic_weighted = area_grid*dC_neg_defor_temperate_oceanic/np.nanmean(dC_neg_defor_temperate_oceanic_area_mask)


dC_temperate_oceanic_mean = (np.nansum(dC_refor_temperate_oceanic*area_grid)+np.nansum(dC_defor_temperate_oceanic*area_grid))/(refor_temperate_oceanic_area+defor_temperate_oceanic_area)


###FRACTION
fraction_refor_temperate_oceanic = dT_refor_temperate_oceanic/dC_refor_temperate_oceanic
fraction_defor_temperate_oceanic = dT_defor_temperate_oceanic/dC_defor_temperate_oceanic
fraction_refor_temperate_oceanic_weighted = dT_refor_temperate_oceanic_weighted/dC_refor_temperate_oceanic_weighted
fraction_defor_temperate_oceanic_weighted = dT_defor_temperate_oceanic_weighted/dC_defor_temperate_oceanic_weighted
fraction_refor_temperate_oceanic_weighted_abs = np.abs(dT_refor_temperate_oceanic_weighted)/np.abs(dC_refor_temperate_oceanic_weighted)
fraction_defor_temperate_oceanic_weighted_abs = np.abs(dT_defor_temperate_oceanic_weighted)/np.abs(dC_defor_temperate_oceanic_weighted)
fraction_neg_refor_temperate_oceanic_weighted = dT_neg_refor_temperate_oceanic_weighted/dC_neg_refor_temperate_oceanic_weighted
fraction_neg_defor_temperate_oceanic_weighted = dT_neg_defor_temperate_oceanic_weighted/dC_neg_defor_temperate_oceanic_weighted

fraction_temperate_oceanic_mean = (np.nansum(fraction_refor_temperate_oceanic*area_grid)+np.nansum(fraction_defor_temperate_oceanic*area_grid))/(refor_temperate_oceanic_area+defor_temperate_oceanic_area)


fraction_temperate_oceanic_weighted_median = np.nanmedian(np.concatenate((fraction_refor_temperate_oceanic_weighted,fraction_defor_temperate_oceanic_weighted),axis=0))
fraction_temperate_oceanic_weighted_median_abs = np.nanmedian(np.concatenate((fraction_refor_temperate_oceanic_weighted_abs,fraction_defor_temperate_oceanic_weighted_abs),axis=0))
fraction_temperate_oceanic_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_temperate_oceanic_weighted_abs,fraction_defor_temperate_oceanic_weighted_abs),axis=0))

#Get rid of outliers
print(np.count_nonzero(np.concatenate((fraction_refor_temperate_oceanic_weighted_abs,fraction_defor_temperate_oceanic_weighted_abs),axis=0)>5)/np.count_nonzero(~np.isnan(np.concatenate((fraction_refor_temperate_oceanic_weighted_abs,fraction_defor_temperate_oceanic_weighted_abs),axis=0))))
fraction_refor_temperate_oceanic_weighted[fraction_refor_temperate_oceanic_weighted_abs>5]=np.nan
fraction_defor_temperate_oceanic_weighted[fraction_defor_temperate_oceanic_weighted_abs>5]=np.nan
fraction_refor_temperate_oceanic_weighted_abs[fraction_refor_temperate_oceanic_weighted_abs>5]=np.nan
fraction_defor_temperate_oceanic_weighted_abs[fraction_defor_temperate_oceanic_weighted_abs>5]=np.nan

fraction_temperate_oceanic_weighted_mean = np.nanmean(np.concatenate((fraction_refor_temperate_oceanic_weighted,fraction_defor_temperate_oceanic_weighted),axis=0))
fraction_temperate_oceanic_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_temperate_oceanic_weighted_abs,fraction_defor_temperate_oceanic_weighted_abs),axis=0))
fraction_temperate_oceanic_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_refor_temperate_oceanic_weighted,fraction_defor_temperate_oceanic_weighted),axis=0),5)
fraction_temperate_oceanic_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_refor_temperate_oceanic_weighted,fraction_defor_temperate_oceanic_weighted),axis=0),95)

fraction_neg_refor_temperate_oceanic_weighted[fraction_neg_refor_temperate_oceanic_weighted<-5]=np.nan
fraction_neg_defor_temperate_oceanic_weighted[fraction_neg_defor_temperate_oceanic_weighted<-5]=np.nan
fraction_neg_refor_temperate_oceanic_weighted[fraction_neg_refor_temperate_oceanic_weighted>0]=np.nan
fraction_neg_defor_temperate_oceanic_weighted[fraction_neg_defor_temperate_oceanic_weighted>0]=np.nan
fraction_neg_refor_temperate_oceanic_weighted_mean = np.nanmean(fraction_neg_refor_temperate_oceanic_weighted)
fraction_neg_defor_temperate_oceanic_weighted_mean = np.nanmean(fraction_neg_defor_temperate_oceanic_weighted)

fraction_neg_temperate_oceanic_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_neg_refor_temperate_oceanic_weighted,fraction_neg_defor_temperate_oceanic_weighted),axis=0),5)
fraction_neg_temperate_oceanic_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_neg_refor_temperate_oceanic_weighted,fraction_neg_defor_temperate_oceanic_weighted),axis=0),95)
fraction_neg_temperate_oceanic_weighted_mean = np.nanmean(np.concatenate((fraction_neg_refor_temperate_oceanic_weighted,fraction_neg_defor_temperate_oceanic_weighted),axis=0))
fraction_neg_temperate_oceanic_weighted_median = np.nanmedian(np.concatenate((fraction_neg_refor_temperate_oceanic_weighted,fraction_neg_defor_temperate_oceanic_weighted),axis=0))


fraction_temperate_oceanic_weighted_std_abs = np.nanstd(np.concatenate((fraction_refor_temperate_oceanic_weighted_abs,fraction_defor_temperate_oceanic_weighted_abs),axis=0))



"""
TEMPERATE_CONTINENTAL
"""
###MEAN
refor_temperate_continental = np.where(refor_local*temperate_continental_mask!=0,refor_local*temperate_continental_mask,np.nan)
defor_temperate_continental = np.where(defor_local*temperate_continental_mask!=0,defor_local*temperate_continental_mask,np.nan)

refor_temperate_continental_area = np.nansum(~np.isnan(refor_temperate_continental)*area_grid)
defor_temperate_continental_area = np.nansum(~np.isnan(defor_temperate_continental)*area_grid)

total_temperate_continental_mean = (np.nansum(refor_temperate_continental*area_grid)+np.nansum(defor_temperate_continental*area_grid))/(refor_temperate_continental_area+defor_temperate_continental_area)


#RANGE
refor_temperate_continental_area_mask = np.where(~np.isnan(refor_temperate_continental),area_grid,np.nan)
refor_temperate_continental_weighted = area_grid*refor_temperate_continental/np.nanmean(refor_temperate_continental_area_mask)
defor_temperate_continental_area_mask = np.where(~np.isnan(defor_temperate_continental),area_grid,np.nan)
defor_temperate_continental_weighted = area_grid*defor_temperate_continental/np.nanmean(defor_temperate_continental_area_mask)
temperate_continental_std = np.nanstd(np.concatenate((refor_temperate_continental_weighted,defor_temperate_continental_weighted),axis=0))

###dT
dT_refor_temperate_continental = np.where(refor_dT*temperate_continental_mask!=0,refor_dT*temperate_continental_mask,np.nan)
dT_defor_temperate_continental = np.where(defor_dT*temperate_continental_mask!=0,defor_dT*temperate_continental_mask,np.nan)
dT_refor_temperate_continental_weighted = refor_temperate_continental_area_mask*dT_refor_temperate_continental/np.nanmean(refor_temperate_continental_area_mask)
dT_defor_temperate_continental_weighted = defor_temperate_continental_area_mask*dT_defor_temperate_continental/np.nanmean(defor_temperate_continental_area_mask)

dT_neg_refor_temperate_continental = np.where(dT_refor_temperate_continental<0,dT_refor_temperate_continental,np.nan)
dT_neg_defor_temperate_continental = np.where(dT_defor_temperate_continental<0,dT_defor_temperate_continental,np.nan)
dT_neg_refor_temperate_continental_area_mask = np.where(~np.isnan(dT_neg_refor_temperate_continental),area_grid,np.nan)
dT_neg_defor_temperate_continental_area_mask = np.where(~np.isnan(dT_neg_defor_temperate_continental),area_grid,np.nan)
dT_neg_refor_temperate_continental_weighted = area_grid*dT_neg_refor_temperate_continental/np.nanmean(dT_neg_refor_temperate_continental_area_mask)
dT_neg_defor_temperate_continental_weighted = area_grid*dT_neg_defor_temperate_continental/np.nanmean(dT_neg_defor_temperate_continental_area_mask)


dT_neg_refor_temperate_continental_area_perc = 100*(np.nansum([dT_refor_temperate_continental<=0]*area_grid)/(refor_temperate_continental_area))
dT_neg_defor_temperate_continental_area_perc = 100*(np.nansum([dT_defor_temperate_continental<=0]*area_grid)/(defor_temperate_continental_area))
dT_neg_temperate_continental_area_perc = 100*(np.nansum([dT_refor_temperate_continental<=0]*area_grid)+np.nansum([dT_defor_temperate_continental<=0]*area_grid))/(refor_temperate_continental_area+defor_temperate_continental_area)


dT_temperate_continental_mean = (np.nansum(dT_refor_temperate_continental*area_grid)+np.nansum(dT_defor_temperate_continental*area_grid))/(refor_temperate_continental_area+defor_temperate_continental_area)
dT_temperate_continental_std = np.nanstd(np.concatenate((dT_refor_temperate_continental_weighted,dT_defor_temperate_continental_weighted),axis=0))

###dC
dC_refor_temperate_continental = np.where(refor_dC*temperate_continental_mask!=0,refor_dC*temperate_continental_mask,np.nan)
dC_defor_temperate_continental = np.where(defor_dC*temperate_continental_mask!=0,defor_dC*temperate_continental_mask,np.nan)
dC_refor_temperate_continental_weighted = refor_temperate_continental_area_mask*dC_refor_temperate_continental/np.nanmean(refor_temperate_continental_area_mask)
dC_defor_temperate_continental_weighted = defor_temperate_continental_area_mask*dC_defor_temperate_continental/np.nanmean(defor_temperate_continental_area_mask)

dC_neg_refor_temperate_continental = np.where(dT_refor_temperate_continental<0,dC_refor_temperate_continental,np.nan)
dC_neg_defor_temperate_continental = np.where(dT_defor_temperate_continental<0,dC_defor_temperate_continental,np.nan)
dC_neg_refor_temperate_continental_area_mask = np.where(~np.isnan(dC_neg_refor_temperate_continental),area_grid,np.nan)
dC_neg_defor_temperate_continental_area_mask = np.where(~np.isnan(dC_neg_defor_temperate_continental),area_grid,np.nan)
dC_neg_refor_temperate_continental_weighted = area_grid*dC_neg_refor_temperate_continental/np.nanmean(dC_neg_refor_temperate_continental_area_mask)
dC_neg_defor_temperate_continental_weighted = area_grid*dC_neg_defor_temperate_continental/np.nanmean(dC_neg_defor_temperate_continental_area_mask)


dC_temperate_continental_mean = (np.nansum(dC_refor_temperate_continental*area_grid)+np.nansum(dC_defor_temperate_continental*area_grid))/(refor_temperate_continental_area+defor_temperate_continental_area)


###FRACTION
fraction_refor_temperate_continental = dT_refor_temperate_continental/dC_refor_temperate_continental
fraction_defor_temperate_continental = dT_defor_temperate_continental/dC_defor_temperate_continental
fraction_refor_temperate_continental_weighted = dT_refor_temperate_continental_weighted/dC_refor_temperate_continental_weighted
fraction_defor_temperate_continental_weighted = dT_defor_temperate_continental_weighted/dC_defor_temperate_continental_weighted
fraction_refor_temperate_continental_weighted_abs = np.abs(dT_refor_temperate_continental_weighted)/np.abs(dC_refor_temperate_continental_weighted)
fraction_defor_temperate_continental_weighted_abs = np.abs(dT_defor_temperate_continental_weighted)/np.abs(dC_defor_temperate_continental_weighted)
fraction_neg_refor_temperate_continental_weighted = dT_neg_refor_temperate_continental_weighted/dC_neg_refor_temperate_continental_weighted
fraction_neg_defor_temperate_continental_weighted = dT_neg_defor_temperate_continental_weighted/dC_neg_defor_temperate_continental_weighted

fraction_temperate_continental_mean = (np.nansum(fraction_refor_temperate_continental*area_grid)+np.nansum(fraction_defor_temperate_continental*area_grid))/(refor_temperate_continental_area+defor_temperate_continental_area)


fraction_temperate_continental_weighted_median = np.nanmedian(np.concatenate((fraction_refor_temperate_continental_weighted,fraction_defor_temperate_continental_weighted),axis=0))
fraction_temperate_continental_weighted_median_abs = np.nanmedian(np.concatenate((fraction_refor_temperate_continental_weighted_abs,fraction_defor_temperate_continental_weighted_abs),axis=0))
fraction_temperate_continental_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_temperate_continental_weighted_abs,fraction_defor_temperate_continental_weighted_abs),axis=0))

#Get rid of outliers
print(np.count_nonzero(np.concatenate((fraction_refor_temperate_continental_weighted_abs,fraction_defor_temperate_continental_weighted_abs),axis=0)>5)/np.count_nonzero(~np.isnan(np.concatenate((fraction_refor_temperate_continental_weighted_abs,fraction_defor_temperate_continental_weighted_abs),axis=0))))
fraction_refor_temperate_continental_weighted[fraction_refor_temperate_continental_weighted_abs>5]=np.nan
fraction_defor_temperate_continental_weighted[fraction_defor_temperate_continental_weighted_abs>5]=np.nan
fraction_refor_temperate_continental_weighted_abs[fraction_refor_temperate_continental_weighted_abs>5]=np.nan
fraction_defor_temperate_continental_weighted_abs[fraction_defor_temperate_continental_weighted_abs>5]=np.nan

fraction_temperate_continental_weighted_mean = np.nanmean(np.concatenate((fraction_refor_temperate_continental_weighted,fraction_defor_temperate_continental_weighted),axis=0))
fraction_temperate_continental_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_temperate_continental_weighted_abs,fraction_defor_temperate_continental_weighted_abs),axis=0))
fraction_temperate_continental_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_refor_temperate_continental_weighted,fraction_defor_temperate_continental_weighted),axis=0),5)
fraction_temperate_continental_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_refor_temperate_continental_weighted,fraction_defor_temperate_continental_weighted),axis=0),95)

fraction_neg_refor_temperate_continental_weighted[fraction_neg_refor_temperate_continental_weighted<-5]=np.nan
fraction_neg_defor_temperate_continental_weighted[fraction_neg_defor_temperate_continental_weighted<-5]=np.nan
fraction_neg_refor_temperate_continental_weighted[fraction_neg_refor_temperate_continental_weighted>0]=np.nan
fraction_neg_defor_temperate_continental_weighted[fraction_neg_defor_temperate_continental_weighted>0]=np.nan
fraction_neg_refor_temperate_continental_weighted_mean = np.nanmean(fraction_neg_refor_temperate_continental_weighted)
fraction_neg_defor_temperate_continental_weighted_mean = np.nanmean(fraction_neg_defor_temperate_continental_weighted)

fraction_neg_temperate_continental_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_neg_refor_temperate_continental_weighted,fraction_neg_defor_temperate_continental_weighted),axis=0),5)
fraction_neg_temperate_continental_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_neg_refor_temperate_continental_weighted,fraction_neg_defor_temperate_continental_weighted),axis=0),95)
fraction_neg_temperate_continental_weighted_mean = np.nanmean(np.concatenate((fraction_neg_refor_temperate_continental_weighted,fraction_neg_defor_temperate_continental_weighted),axis=0))
fraction_neg_temperate_continental_weighted_median = np.nanmedian(np.concatenate((fraction_neg_refor_temperate_continental_weighted,fraction_neg_defor_temperate_continental_weighted),axis=0))


fraction_temperate_continental_weighted_std_abs = np.nanstd(np.concatenate((fraction_refor_temperate_continental_weighted_abs,fraction_defor_temperate_continental_weighted_abs),axis=0))






"""
BOREAL
"""
###MEAN
refor_boreal = np.where(refor_local*boreal_mask!=0,refor_local*boreal_mask,np.nan)
defor_boreal = np.where(defor_local*boreal_mask!=0,defor_local*boreal_mask,np.nan)

refor_boreal_area = np.nansum(~np.isnan(refor_boreal)*area_grid)
defor_boreal_area = np.nansum(~np.isnan(defor_boreal)*area_grid)

total_boreal_mean = (np.nansum(refor_boreal*area_grid)+np.nansum(defor_boreal*area_grid))/(refor_boreal_area+defor_boreal_area)

#RANGE
refor_boreal_area_mask = np.where(~np.isnan(refor_boreal),area_grid,np.nan)
refor_boreal_weighted = area_grid*refor_boreal/np.nanmean(refor_boreal_area_mask)
defor_boreal_area_mask = np.where(~np.isnan(defor_boreal),area_grid,np.nan)
defor_boreal_weighted = area_grid*defor_boreal/np.nanmean(defor_boreal_area_mask)
boreal_std = np.nanstd(np.concatenate((refor_boreal_weighted,defor_boreal_weighted),axis=0))



###dT
dT_refor_boreal = np.where(refor_dT*boreal_mask!=0,refor_dT*boreal_mask,np.nan)
dT_defor_boreal = np.where(defor_dT*boreal_mask!=0,defor_dT*boreal_mask,np.nan)
dT_refor_boreal_weighted = refor_boreal_area_mask*dT_refor_boreal/np.nanmean(refor_boreal_area_mask)
dT_defor_boreal_weighted = defor_boreal_area_mask*dT_defor_boreal/np.nanmean(defor_boreal_area_mask)

dT_neg_refor_boreal = np.where(dT_refor_boreal<0,dT_refor_boreal,np.nan)
dT_neg_defor_boreal = np.where(dT_defor_boreal<0,dT_defor_boreal,np.nan)
dT_neg_refor_boreal_area_mask = np.where(~np.isnan(dT_neg_refor_boreal),area_grid,np.nan)
dT_neg_defor_boreal_area_mask = np.where(~np.isnan(dT_neg_defor_boreal),area_grid,np.nan)
dT_neg_refor_boreal_weighted = area_grid*dT_neg_refor_boreal/np.nanmean(dT_neg_refor_boreal_area_mask)
dT_neg_defor_boreal_weighted = area_grid*dT_neg_defor_boreal/np.nanmean(dT_neg_defor_boreal_area_mask)


dT_neg_refor_boreal_area_perc = 100*(np.nansum([dT_refor_boreal<=0]*area_grid)/(refor_boreal_area))
dT_neg_defor_boreal_area_perc = 100*(np.nansum([dT_defor_boreal<=0]*area_grid)/(defor_boreal_area))
dT_neg_boreal_area_perc = 100*(np.nansum([dT_refor_boreal<=0]*area_grid)+np.nansum([dT_defor_boreal<=0]*area_grid))/(refor_boreal_area+defor_boreal_area)

dT_boreal_mean = (np.nansum(dT_refor_boreal*area_grid)+np.nansum(dT_defor_boreal*area_grid))/(refor_boreal_area+defor_boreal_area)
dT_boreal_std = np.nanstd(np.concatenate((dT_refor_boreal_weighted,dT_defor_boreal_weighted),axis=0))


###dC
dC_refor_boreal = np.where(refor_dC*boreal_mask!=0,refor_dC*boreal_mask,np.nan)
dC_defor_boreal = np.where(defor_dC*boreal_mask!=0,defor_dC*boreal_mask,np.nan)
dC_refor_boreal_weighted = refor_boreal_area_mask*dC_refor_boreal/np.nanmean(refor_boreal_area_mask)
dC_defor_boreal_weighted = defor_boreal_area_mask*dC_defor_boreal/np.nanmean(defor_boreal_area_mask)

dC_boreal_mean = (np.nansum(dC_refor_boreal*area_grid)+np.nansum(dC_defor_boreal*area_grid))/(refor_boreal_area+defor_boreal_area)

dC_neg_refor_boreal = np.where(dT_refor_boreal<0,dC_refor_boreal,np.nan)
dC_neg_defor_boreal = np.where(dT_defor_boreal<0,dC_defor_boreal,np.nan)
dC_neg_refor_boreal_area_mask = np.where(~np.isnan(dC_neg_refor_boreal),area_grid,np.nan)
dC_neg_defor_boreal_area_mask = np.where(~np.isnan(dC_neg_defor_boreal),area_grid,np.nan)
dC_neg_refor_boreal_weighted = area_grid*dC_neg_refor_boreal/np.nanmean(dC_neg_refor_boreal_area_mask)
dC_neg_defor_boreal_weighted = area_grid*dC_neg_defor_boreal/np.nanmean(dC_neg_defor_boreal_area_mask)

###FRACTION
fraction_refor_boreal = dT_refor_boreal/dC_refor_boreal
fraction_defor_boreal = dT_defor_boreal/dC_defor_boreal
fraction_refor_boreal_weighted = dT_refor_boreal_weighted/dC_refor_boreal_weighted
fraction_defor_boreal_weighted = dT_defor_boreal_weighted/dC_defor_boreal_weighted
fraction_refor_boreal_weighted_abs = np.abs(dT_refor_boreal_weighted)/np.abs(dC_refor_boreal_weighted)
fraction_defor_boreal_weighted_abs = np.abs(dT_defor_boreal_weighted)/np.abs(dC_defor_boreal_weighted)
fraction_neg_refor_boreal_weighted = dT_neg_refor_boreal_weighted/dC_neg_refor_boreal_weighted
fraction_neg_defor_boreal_weighted = dT_neg_defor_boreal_weighted/dC_neg_defor_boreal_weighted


fraction_boreal_mean = (np.nansum(fraction_refor_boreal*area_grid)+np.nansum(fraction_defor_boreal*area_grid))/(refor_boreal_area+defor_boreal_area)

fraction_boreal_weighted_median = np.nanmedian(np.concatenate((fraction_refor_boreal_weighted,fraction_defor_boreal_weighted),axis=0))
fraction_boreal_weighted_median_abs = np.nanmedian(np.concatenate((fraction_refor_boreal_weighted_abs,fraction_defor_boreal_weighted_abs),axis=0))

#Get rid of outliers
print(np.count_nonzero(np.concatenate((fraction_refor_boreal_weighted_abs,fraction_defor_boreal_weighted_abs),axis=0)>5)/np.count_nonzero(~np.isnan(np.concatenate((fraction_refor_boreal_weighted_abs,fraction_defor_boreal_weighted_abs),axis=0))))
fraction_refor_boreal_weighted[fraction_refor_boreal_weighted_abs>5]=np.nan
fraction_defor_boreal_weighted[fraction_defor_boreal_weighted_abs>5]=np.nan
fraction_refor_boreal_weighted_abs[fraction_refor_boreal_weighted_abs>5]=np.nan
fraction_defor_boreal_weighted_abs[fraction_defor_boreal_weighted_abs>5]=np.nan

fraction_boreal_weighted_mean = np.nanmean(np.concatenate((fraction_refor_boreal_weighted,fraction_defor_boreal_weighted),axis=0))
fraction_boreal_weighted_mean_abs = np.nanmean(np.concatenate((fraction_refor_boreal_weighted_abs,fraction_defor_boreal_weighted_abs),axis=0))
fraction_boreal_weighted_std_abs = np.nanstd(np.concatenate((fraction_refor_boreal_weighted_abs,fraction_defor_boreal_weighted_abs),axis=0))
fraction_boreal_weighted_std = np.nanstd(np.concatenate((fraction_refor_boreal_weighted,fraction_defor_boreal_weighted),axis=0))
fraction_boreal_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_refor_boreal_weighted,fraction_defor_boreal_weighted),axis=0),5)
fraction_boreal_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_refor_boreal_weighted,fraction_defor_boreal_weighted),axis=0),95)


fraction_neg_refor_boreal_weighted[fraction_neg_refor_boreal_weighted<-5]=np.nan
fraction_neg_defor_boreal_weighted[fraction_neg_defor_boreal_weighted<-5]=np.nan
fraction_neg_refor_boreal_weighted[fraction_neg_refor_boreal_weighted>0]=np.nan
fraction_neg_defor_boreal_weighted[fraction_neg_defor_boreal_weighted>0]=np.nan
fraction_neg_refor_boreal_weighted_mean = np.nanmean(fraction_neg_refor_boreal_weighted)
fraction_neg_defor_boreal_weighted_mean = np.nanmean(fraction_neg_defor_boreal_weighted)

fraction_neg_boreal_weighted_perc05 = np.nanpercentile(np.concatenate((fraction_neg_refor_boreal_weighted,fraction_neg_defor_boreal_weighted),axis=0),5)
fraction_neg_boreal_weighted_perc95 = np.nanpercentile(np.concatenate((fraction_neg_refor_boreal_weighted,fraction_neg_defor_boreal_weighted),axis=0),95)
fraction_neg_boreal_weighted_mean = np.nanmean(np.concatenate((fraction_neg_refor_boreal_weighted,fraction_neg_defor_boreal_weighted),axis=0))
fraction_neg_boreal_weighted_median = np.nanmedian(np.concatenate((fraction_neg_refor_boreal_weighted,fraction_neg_defor_boreal_weighted),axis=0))


fraction_neg_boreal_weighted_std = np.nanstd(np.concatenate((fraction_neg_refor_boreal_weighted,fraction_neg_defor_boreal_weighted),axis=0))




0.0011491486206334294
0.011471752000714009
0.009594047774441979
0.009778045059191696
0.0031251720909741725


In [23]:
textfig3summer = f"In boreal summer, all temperate ({fraction_temperate_weighted_perc05*100:.1f}% - {fraction_temperate_weighted_perc95*100:.1f}%; median {fraction_temperate_weighted_median*100:.1f}%) and boreal ({fraction_boreal_weighted_perc05*100:.1f}% - {fraction_boreal_weighted_perc95*100:.1f}%; median {fraction_boreal_weighted_median*100:.1f}%) regions experience a BGP induced local cooling of forests."

print(textfig3summer)

In boreal summer, all temperate (8.7% - 57.2%; median 19.8%) and boreal (4.0% - 41.9%; median 18.0%) regions experience a BGP induced local cooling of forests.


In [4]:
#calculate climate zone means

import numpy as np

#TROPICAL


#NEW AREA SPECIFIC
refor_tropical = np.where(refor_local*tropical_mask!=0,refor_local*tropical_mask,np.nan)
defor_tropical = np.where(defor_local*tropical_mask!=0,defor_local*tropical_mask,np.nan)

refor_tropical_area = np.nansum(~np.isnan(refor_tropical)*area_grid)
defor_tropical_area = np.nansum(~np.isnan(defor_tropical)*area_grid)


total_tropical_mean = (np.nansum(refor_tropical*area_grid)+np.nansum(defor_tropical*area_grid))/(refor_tropical_area+defor_tropical_area)
defor_tropical_mean = (np.nansum(defor_tropical*area_grid))/(defor_tropical_area)
refor_tropical_mean = (np.nansum(refor_tropical*area_grid))/(refor_tropical_area)

#RANGE
refor_tropical_area_mask = np.where(~np.isnan(refor_tropical),area_grid,np.nan)
refor_tropical_weighted = area_grid*refor_tropical/np.nanmean(refor_tropical_area_mask)
defor_tropical_area_mask = np.where(~np.isnan(defor_tropical),area_grid,np.nan)
defor_tropical_weighted = area_grid*defor_tropical/np.nanmean(defor_tropical_area_mask)
tropical_std = np.nanstd(np.concatenate((refor_tropical_weighted,defor_tropical_weighted),axis=0))
tropical_defor_std = np.nanstd(defor_tropical_weighted)
tropical_refor_std = np.nanstd(refor_tropical_weighted)


#TEMPERATE

###NEW AREA SPECIFIC
refor_temperate = np.where(refor_local*temperate_mask!=0,refor_local*temperate_mask,np.nan)
defor_temperate = np.where(defor_local*temperate_mask!=0,defor_local*temperate_mask,np.nan)

refor_temperate_area = np.nansum(~np.isnan(refor_temperate)*area_grid)
defor_temperate_area = np.nansum(~np.isnan(defor_temperate)*area_grid)

total_temperate_mean = (np.nansum(refor_temperate*area_grid)+np.nansum(defor_temperate*area_grid))/(refor_temperate_area+defor_temperate_area)
defor_temperate_mean = (np.nansum(defor_temperate*area_grid))/(defor_temperate_area)
refor_temperate_mean = (np.nansum(refor_temperate*area_grid))/(refor_temperate_area)

#RANGE
refor_temperate_area_mask = np.where(~np.isnan(refor_temperate),area_grid,np.nan)
refor_temperate_weighted = area_grid*refor_temperate/np.nanmean(refor_temperate_area_mask)
defor_temperate_area_mask = np.where(~np.isnan(defor_temperate),area_grid,np.nan)
defor_temperate_weighted = area_grid*defor_temperate/np.nanmean(defor_temperate_area_mask)
temperate_std = np.nanstd(np.concatenate((refor_temperate_weighted,defor_temperate_weighted),axis=0))
temperate_defor_std = np.nanstd(defor_temperate_weighted)
temperate_refor_std = np.nanstd(refor_temperate_weighted)




#BOREAL

###NEW AREA SPECIFIC
refor_boreal = np.where(refor_local*boreal_mask!=0,refor_local*boreal_mask,np.nan)
defor_boreal = np.where(defor_local*boreal_mask!=0,defor_local*boreal_mask,np.nan)

refor_boreal_area = np.nansum(~np.isnan(refor_boreal)*area_grid)
defor_boreal_area = np.nansum(~np.isnan(defor_boreal)*area_grid)

total_boreal_mean = (np.nansum(refor_boreal*area_grid)+np.nansum(defor_boreal*area_grid))/(refor_boreal_area+defor_boreal_area)
defor_boreal_mean = (np.nansum(defor_boreal*area_grid))/(defor_boreal_area)
refor_boreal_mean = (np.nansum(refor_boreal*area_grid))/(refor_boreal_area)


#RANGE
refor_boreal_area_mask = np.where(~np.isnan(refor_boreal),area_grid,np.nan)
refor_boreal_weighted = area_grid*refor_boreal/np.nanmean(refor_boreal_area_mask)
defor_boreal_area_mask = np.where(~np.isnan(defor_boreal),area_grid,np.nan)
defor_boreal_weighted = area_grid*defor_boreal/np.nanmean(defor_boreal_area_mask)
boreal_std = np.nanstd(np.concatenate((refor_boreal_weighted,defor_boreal_weighted),axis=0))
boreal_defor_std = np.nanstd(defor_boreal_weighted)
boreal_refor_std = np.nanstd(refor_boreal_weighted)

"""
CHECK DEFOR VS REFOR 
"""
refor_perc = np.nanpercentile(refor_local,95)
total_area = np.sum(area_grid[~np.isnan(refor_local)])


area_above_reforperc = np.sum(area_grid[refor_local>=refor_perc ])
relative_area = 100*area_above_reforperc/total_area
total_area = np.sum(area_grid[~np.isnan(defor_local)])


area_above_deforperc = np.sum(area_grid[defor_local>=refor_perc ])
relative_area = 100*area_above_deforperc/total_area

                         
print("tropical defor:", defor_tropical_mean,"+/-",tropical_defor_std)
print("temperate defor:",defor_temperate_mean,"+/-",temperate_defor_std)
print("boreal defor:",defor_boreal_mean,"+/-",boreal_defor_std)

print("tropical refor:", refor_tropical_mean,"+/-",tropical_refor_std)
print("temperate refor:",refor_temperate_mean,"+/-",temperate_refor_std)
print("boreal refor:",refor_boreal_mean,"+/-",boreal_refor_std)

tropical defor: 790.1975 +/- 105.628845
temperate defor: 254.89485 +/- 115.851074
boreal defor: 213.05263 +/- 43.550854
tropical refor: 523.10144 +/- 180.04956
temperate refor: 166.97827 +/- 104.062935
boreal refor: 102.73 +/- 71.7374


In [5]:
textfig41 = f"More than a third ({relative_area:.1f}%, {area_above_deforperc/1000000:.0f} Mha) of all current forest cover exerts an equal or higher impact on climate in case of deforestation than the 95th percentile ({refor_perc:.1f} tCO2eq/ha, {area_above_reforperc/1000000:.0f} Mha) effect of potential re- and afforestation sites (Fig 4.). Due to the relative scarcity of optimal re- and afforestation sites large-scale efforts, like the Bonn Challenge24 (350Mha), will also need to consider much less effective sites (Fig 4.)."  
textfig42 = f"The high BGP and BGC impact of tropical forests on climate suggests that future intensification of land use in these areas will have a stronger adverse effect than historic deforestation of temperate forests (tropic mean of standing forests {defor_tropical_mean:.1f} tCO2eq/ha. (σ {tropical_defor_std:.1f} tCO2eq/ha) vs. temperate re- and afforestation sites {refor_temperate_mean:.1f} tCO2eq/ha (σ {temperate_refor_std:.1f} tCO2eq/ha)). "

print(textfig41,textfig42)

More than a third (36.6%, 1058 Mha) of all current forest cover exerts an equal or higher impact on climate in case of deforestation than the 95th percentile (601.4 tCO2eq/ha, 117 Mha) effect of potential re- and afforestation sites (Fig 4.). Due to the relative scarcity of optimal re- and afforestation sites large-scale efforts, like the Bonn Challenge24 (350Mha), will also need to consider much less effective sites (Fig 4.). The high BGP and BGC impact of tropical forests on climate suggests that future intensification of land use in these areas will have a stronger adverse effect than historic deforestation of temperate forests (tropic mean of standing forests 790.2 tCO2eq/ha. (σ 105.6 tCO2eq/ha) vs. temperate re- and afforestation sites 167.0 tCO2eq/ha (σ 104.1 tCO2eq/ha)). 
